## Build usable data for VEX functions

### Goal structure
```yaml
foo:
  added: '14.0'
  help: <html>foo</html>
  sigs:
  - args:
    - int
    argnames:
    - bar
    return:
    - float
    contexts:
    - cvex
    variadic: true
    variadic_pair: true
    external: true
    fillstate: EXACT
```

In [3]:
import re
import bs4
import copy
import json
import pickle
import random
import zipfile
import requests
import os.path as op
from pprint import pprint
from subprocess import run, PIPE
from IPython.display import HTML, Markdown

### Fetch data from VCC

#### Structure
```yaml
foo:
  sigs:
  - args:
    - int
    return:
    - float
    contexts:
    - cvex
    variadic: true
    variadic_pair: true
    external: true
```

In [4]:
proc = run(['vcc', '--list-context-json'],
           shell=True, stdout=PIPE)

contexts = json.loads(proc.stdout)

vcc = {}

for con in contexts:
    proc = run(['vcc', '--list-context-json', con],
               shell=True, stdout=PIPE)
    funs = json.loads(proc.stdout)['functions']

    # Discard deprecated signatures.
    funs = {f: [s for s in sigs if 'deprecated' not in s]
            for f, sigs in funs.items()}
    
    # Include context signatures.
    for f, sigs in funs.items():
        for s in sigs:
            s['contexts'] = [con,]

    # Add unique functions and signatures to common dictionary.
    for f in funs:
        # Function not exists.
        if f not in vcc:
            vcc[f] = {'sigs': funs[f]}
            continue

        # Function exists. Search for unique signatures.
        for con_s in funs[f]:
            for s in vcc[f]['sigs']:
                if (con_s['args'], con_s['return']) == (s['args'], s['return']):
                    s['contexts'].append(con)
                    break  # Suppress for-else clause.
            # Non-unique not found (signature is unique).
            else:
                vcc[f]['sigs'].append(con_s)

def test():
    f = random.choice(list(vcc))
    print(
        'Total functions: %d' % len(vcc),
        '\nTotal signatures: %d' % sum(len(vcc[f]['sigs']) for f in vcc),
        '\nRandom function: %s' % f,
        '\nSignatures: %d' % len(vcc[f]['sigs']),
    )
    pprint(vcc[f])

test()

Total functions: 720 
Total signatures: 3233 
Random function: primhedge 
Signatures: 2
{'sigs': [{'args': ['const int', 'const int'],
           'contexts': ['surface',
                        'displace',
                        'light',
                        'shadow',
                        'fog',
                        'chop',
                        'pop',
                        'sop',
                        'cop2',
                        'image3d',
                        'cvex'],
           'external': True,
           'return': 'int'},
          {'args': ['const string', 'const int'],
           'contexts': ['surface',
                        'displace',
                        'light',
                        'shadow',
                        'fog',
                        'chop',
                        'pop',
                        'sop',
                        'cop2',
                        'image3d',
                        'cvex'],
           'external': True,
  

### Parse VEX argument names from Houdini help

Extracts data from documentation. It's quite noisy (22% not parsed) and
not full, but documented functions usually filled with argument names
which then can be used to generate high quality completions.

#### Partial data from docs
```yaml
foo:
  sigs:
  - argnames:
    - bar
```

In [5]:
# Parser mini-library.
def is_vex_type(some_type):
    '''Return True if some_type is a valid VEX type name.'''
    return some_type.strip('&[] ') in ['void', 'int', 'float', 'string',
                                       'vector2', 'vector',  'vector4',
                                       'matrix2', 'matrix3', 'matrix',
                                       'bsdf', 'light', 'material']


def arg_data(raw_arg):
    '''
    Parse input string of argument type and name pair. Return argument's
    type and name obtained from the input string.

    Examples:
        'int foo'     -> 'const int', 'foo'
        'int bar&'    -> 'int', 'bar'
        'float baz[]' -> 'float[]', 'baz'
    '''
    # Simplify string.
    arg = raw_arg.replace('&', ' ')
    arg = arg.replace('[]', ' ')
    arg = arg.replace('const ', ' ')
    arg = arg.replace('export ', ' ')
    arg = [s.strip() for s in arg.split()]

    # Not a type and name pair.
    if len(arg) != 2:
        return

    # Not a valid type.
    if not is_vex_type(arg[0]):
        return

    # Not a valid name.
    if not re.match(r'^\w+(="\w*?"|=[\d.]+)?$', arg[1]):
        return

    # Return array modifier to the type.
    if '[]' in raw_arg:
        arg[0] += '[]'

    # Set const or export modifiers.
    if 'export' in raw_arg:
        arg[0] = 'export ' + arg[0]
    elif 'const ' in raw_arg or '&' not in raw_arg:
        arg[0] = 'const ' + arg[0]

    return arg


def sig_data(raw_sig):
    '''
    Parse input string of function signature. Find function's type, name,
    function arguments's types and names. Return as dictionary with single
    item containing function name and single-item list of signature data.

    Example:
        'string foo(string bar, int baz[])'
        ->
        'foo': [{'argnames': ['bar', 'baz'],
                 'args': ['const string','const int[]'],
                 'return': 'string'}]
    '''
    #Replace common documentation flaws to make life easier.
    sig = raw_sig.replace(';', ',')
    sig = sig.replace("'", '"')
    sig = sig.replace('matrix4', 'matrix')
    sig = sig.replace('vector3', 'vector')
    sig = sig.replace(', ...', '')

    # Retrieve data.
    match = re.match(r'(\w+(?:\s?\[\])?)?\s*(\b\w+\b)\(\s*(.+)\s*\)', sig)
    if not match:
        return
    ret, name, args = match.groups()

    # Check return type.
    if ret is None:
        ret = 'void'
    ret = ret.replace(' ', '')
    if not is_vex_type(ret):
        return

    # Parse arguments.
    args = [arg_data(a) for a in args.split(',')]
    if not all(args):
        return

    atypes, anames = zip(*args)
    return {name : [{'return' : ret, 
                     'args' : list(atypes), 
                     'argnames' : list(anames)}]}


def parse_sigs(raw_sigs):
    '''
    Parse each signature string. Merge multiple signatures of same function
    into a dictionary of function names as keys and lists of signature datas
    as values.
    '''
    sigs = {}
    for raw_sig in raw_sigs:
        data = sig_data(raw_sig)
        if not data:
            continue

        name, data = data.popitem()
        if name in sigs:
            sigs[name] += data
        else:
            sigs[name] = data

    # Wrap into another dict, following general structure.
    sigs = {name: {'sigs': sigs[name]} for name in sigs}
    return sigs


def test():
    pprint(parse_sigs(['a(string foo &, int bar[])',
                       'string a(string foo, int bar[])',
                       'vector2 b(float foo, float bar)',
                       'foo bar(baz)']))
    
test()

{'a': {'sigs': [{'argnames': ['foo', 'bar'],
                 'args': ['string', 'const int[]'],
                 'return': 'void'},
                {'argnames': ['foo', 'bar'],
                 'args': ['const string', 'const int[]'],
                 'return': 'string'}]},
 'b': {'sigs': [{'argnames': ['foo', 'bar'],
                 'args': ['const float', 'const float'],
                 'return': 'vector2'}]}}


In [9]:
overrides = '''float       abs(float value)
int         adddetailattrib(int geohandle, string name, float defvalue)
int         addpointattrib(int geohandle, string name, float defvalue)
void        addprim(int primnum; const int geohandle; const string type; const int pt0; int vertices[])
void        addprim(int primnum; const int geohandle; const string type; const int pt0; const int pt1; int vertices[])
void        addprim(int primnum; const int geohandle; const string type; const int pt0; const int pt1; const int pt2; int vertices[])
void        addprim(int primnum; const int geohandle; const string type; const int pt0; const int pt1; const int pt2; const int pt3; int vertices[])
void        addprim(int primnum; const int geohandle; const string type; const int[] points; int vertices[])
int         addprimattrib(int geohandle, string name, float defvalue)
int         addvertexattrib(int geohandle, string name, float defvalue)
float       asin(float value)
void        assign(float & c1, float & c2, vector2 source)
float       attrib(int input, string attribclass, string name, int elemnum)
float       attrib(string geometry, string attribclass, string name, int elemnum)
string      attribtypeinfo(int opinput, string attribute_class, string attribute_name)
string      attribtypeinfo(string geometry, string attribute_class, string attribute_name)
float       avg(float[] values)
vector2     avg(vector2 a, vector2 b, ...)
vector      blackbody(float temperature, float luminance)
void        bumpmapA(string filename, float & du, float & dv, float u, float v)
void        bumpmapA(string filename, float & du, float & dv, vector uvw)
void        bumpmapB(string filename, float & du, float & dv, float u, float v)
void        bumpmapB(string filename, float & du, float & dv, vector uvw)
void        bumpmapG(string filename, float & du, float & dv, float u, float v)
void        bumpmapG(string filename, float & du, float & dv, vector uvw)
void        bumpmapL(string filename, float & du, float & dv, float u, float v)
void        bumpmapL(string filename, float & du, float & dv, vector uvw)
void        bumpmapR(string filename, float & du, float & dv, float u, float v)
void        bumpmapR(string filename, float & du, float & dv, vector uvw)
float       cbrt(float value)
float       ceil(float value)
vector4     chp(string channel)
vector4     chp(string channel, float time)
float       ckspline(float t, float v0, float k0)
vector      ckspline(float t, vector v0, float k0)
vector4     ckspline(float t, vector4 v0, float k0)
vector2     clamp(vector2 value, vector2 min, vector2 max)
vector      colormap(string filename, vector uv0, vector uv1, vector uv2, vector uv3, int samples)
bsdf        cone(vector foo, vector bar, float baz, ...)
float       cos(float value)
float       cosh(float value)
float[]     create_cdf(float pdf[])
float[]     create_pdf(float values[])
vector      cross(vector v1, vector v2)
vector      ctransform(string fromspace, string tospace, vector clr)
float       degrees(float angle)
float       detail(int opinput, string attribute_name)
float       detail(int opinput, string attribute_name, int ignored)
float       detail(string geometry, string attribute_name)
float       detail(string geometry, string attribute_name, int ignored)
float       detailattrib(int opinput, string attribute_name, int ignored, int & success)
float       detailattrib(string geometry, string attribute_name, int ignored, int & success)
int         detailattribsize(int opinput, string attribute_name)
int         detailattribtype(int opinput, string attribute_name)
float       detailintrinsic(int opinput, string intrinsic)
float       detailintrinsic(string geometry, string intrinsic)
float       determinant(matrix mat)
bsdf        diffuse(float roughness, ...)
bsdf        diffuse(vector nml, float roughness, ...)
int         dimport(string name, float & value)
float       distance(vector pos1, vector pos2)
float       distance(vector2 pos1, vector2 pos2)
float       distance(vector4 pos1, vector4 pos2)
float       distance2(vector pos1, vector pos2)
float       distance2(vector2 pos1, vector2 pos2)
float       distance2(vector4 pos1, vector4 pos2)
int         dsmpixel(string map, string channel, int x, int y, float[] & values)
vector      Du(vector value)
vector      Dv(vector value)
vector2     efit(vector2 value, vector2 omin, vector2 omax, vector2 nmin, vector2 nmax)
vector2     efit(vector2 value, vector2 omin, vector2 omax, float nmin, float nmax)
vector      efit(vector value, vector omin, vector omax, float nmin, float nmax)
vector4     efit(vector4 value, vector4 omin, vector4 omax, float nmin, float nmax)
float       exp(float value)
string      expand_udim(float u, float v, string path)
int         findattribval(int opinput, string class, string attribute_name, int value)
int         findattribval(string geometry, string class, string attribute_name, int value)
vector2     fit(vector2 value, vector2 omin, vector2 omax, vector2 nmin, vector nmax)
vector2     fit(vector2 value, vector2 omin, vector2 omax, vector2 nmin, vector2 nmax)
vector2     fit01(vector2 value, vector2 nmin, vector2 nmax)
vector2     fit10(vector2 value, vector2 nmin, vector2 nmax)
vector2     fit11(vector2 value, vector2 nmin, vector2 nmax)
float       floor(float value)
float       frac(float value)
vector      fromNDC(string camera_name, vector point)
vector      fromNDC(vector point)
float       fuzzy_and(float v1, float v2, ...)
int         fuzzy_and(int v1, int v2, ...)
float       fuzzy_nand(float v1, float v2, ...)
int         fuzzy_nand(int v1, int v2, ...)
float       fuzzy_nor(float v1, float v2, ...)
int         fuzzy_nor(int v1, int v2, ...)
float       fuzzy_not(float v)
int         fuzzy_not(int v)
float       fuzzy_nxor(float v1, float v2, ...)
int         fuzzy_nxor(int v1, int v2, ...)
float       fuzzy_or(float v1, float v2, ...)
int         fuzzy_or(int v1, int v2, ...)
float       fuzzy_xor(float v1, float v2, ...)
int         fuzzy_xor(int v1, int v2, ...)
float       getattrib(int input, string attribclass, string name, int elemnum, int & success)
float       getattrib(string geometry, string attribclass, string name, int elemnum, int & success)
void        getbbox(string geometry, vector & min, vector & max)
vector      getblurP(float delta)
bsdf        getcomp(bsdf[] array, int index)
int         getcomp(int[] array, int index)
light       getcomp(light[] array, int index)
material    getcomp(material[] array, int index)
matrix2     getcomp(matrix2[] array, int index)
matrix3     getcomp(matrix3[] array, int index)
matrix      getcomp(matrix[] array, int index)
string      getcomp(string array, int index)
string      getcomp(string[] array, int index)
float       getcomp(vector vec, int index)
vector2     getcomp(vector2[] array, int index)
vector4     getcomp(vector4[] array, int index)
vector      getcomp(vector[] array, int index)
int[]       getlights(vector pos, ...)
void        getpointbbox(string geometry, vector & min, vector & max)
vector      getpointbbox_center(int opinput)
vector      getpointbbox_center(int opinput, string pointgroup)
vector      getpointbbox_center(string filename, string pointgroup)
vector      getpointbbox_max(int opinput)
vector      getpointbbox_max(int opinput, string pointgroup)
vector      getpointbbox_max(string filename, string pointgroup)
vector      getpointbbox_min(int opinput)
vector      getpointbbox_min(int opinput, string pointgroup)
vector      getpointbbox_min(string filename, string pointgroup)
vector      getpointbbox_size(int opinput)
vector      getpointbbox_size(int opinput, string pointgroup)
vector      getpointbbox_size(string filename, string pointgroup)
int         getsmoothP(vector smoothP, vector foo)
matrix      getspace(string fromspace, string tospace)
bsdf        hair(vector N, vector tip, float lobe_shift, float lobe_width_lon, float lobe_with_azi, float glint_shift, float glint_intensity, ...)
int         hasdetailattrib(int opinput, string attribute_name)
int         hasprimattrib(int opinput, string attribute_name)
int         hasvertexattrib(int opinput, string attribute_name)
vector      hsvtorgb(vector hsv)
string      ichname(int planeindex, int chindex, int input)
int         import(string attrib_name, float & result, int input)
int         intersect_all(int input, string group, vector orig, vector dir, vector[] & pos, int[] & prim, vector[] & uvw)
int         intersect_all(int input, string group, vector orig, vector dir, vector[] & pos, int[] & prim, vector[] & uvw, float tol=0.01, float ttol=0.01)
int         intersect_all(int input, vector orig, vector dir, vector[] & pos, int[] & prim, vector[] & uvw)
int         intersect_all(int input, vector orig, vector dir, vector[] & pos, int[] & prim, vector[] & uvw, float tol=0.01, float ttol=0.01)
int         intersect_all(string filename, string group, vector orig, vector dir, vector[] & pos, int[] & prim, vector[] & uvw)
int         intersect_all(string filename, string group, vector orig, vector dir, vector[] & pos, int[] & prim, vector[] & uvw, float tol=0.01, float ttol=0.01)
int         intersect_all(string filename, vector orig, vector dir, vector[] & pos, int[] & prim, vector[] & uvw)
int         intersect_all(string filename, vector orig, vector dir, vector[] & pos, int[] & prim, vector[] & uvw, float tol=0.01, float ttol=0.01)
matrix      invert(matrix mat)
int         isvarying(float variable)
int         len(float[] array)
float       length(float vec)
float       length2(vector vec)
bsdf        lerp(bsdf value1, bsdf value2, float amount)
vector2     lerp(vector2 value1, vector2 value2, float amount)
vector2     lerp(vector2 value1, vector2 value2, vector2 amount)
float       lkspline(float t, float v1, float k1, ...)
vector      lkspline(float t, vector v1, float k1, ...)
vector4     lkspline(float t, vector4 v1, float k1, ...)
float       log(float value)
float       luminance(float r, float g, float b)
float       luminance(vector vec)
matrix      maketransform(int trs, int xyz, vector t, vector r, vector s)
matrix      maketransform(int trs, int xyz, vector t, vector r, vector s, vector p)
matrix3     maketransform(vector zaxis, vector yaxis)
matrix      maketransform(vector zaxis, vector yaxis, vector translate)
int         max(int value1, int value2, ...)
float       max(vector vec)
vector2     max(vector2 value1, vector2 value2, ...)
float       max(vector2 vec)
int         metaimport(int handle, string attribute_name, vector pos, float & value)
float[]     metaimport(string geometry, string attribute_name, vector pos)
int         metanext(int handle)
int         min(int value1, int value2, ...)
float       min(vector vec)
vector2     min(vector2 value1, vector2 value2, ...)
float       min(vector2 vec)
vector      minpos(int opinput, string primgroup, vector pt)
vector      minpos(int opinput, string primgroup, vector pt, float maxdist)
vector      minpos(int opinput, vector pt)
vector      minpos(int opinput, vector pt, float maxdist)
int         nametoprim(string geometry, string name)
int         nearpoint(int opinput, string ptgroup, vector pt)
int         nearpoint(int opinput, string ptgroup, vector pt, float maxdist)
int         nearpoint(int opinput, vector pt)
int         nearpoint(int opinput, vector pt, float maxdist)
int[]       nearpoints(int opinput, string ptgroup, vector pt, float maxdist)
int[]       nearpoints(int opinput, string ptgroup, vector pt, float maxdist, int maxpts)
int[]       nearpoints(int opinput, vector pt, float maxdist)
int[]       nearpoints(int opinput, vector pt, float maxdist, int maxpts)
matrix3     ninput(int input_number, int planeindex, int arrayindex, int component, float u, float v, int frame, ...)
matrix3     ninput(int input_number, int planeindex, int arrayindex, int component, int u, int v, int frame, ...)
vector      normalize(vector v)
vector      ntransform(string fromspace, string tospace, vector n)
vector      ntransform(string tospace, vector n)
vector      ntransform(vector n, matrix transform_matrix)
int         ocio_import(string space; string property; int &value)
vector      ocio_transform(string src; string dest; vector clr)
vector4     ocio_transform(string src; string dest; vector4 clr)
void        opend(int operation_id)
int         ord(string value)
int         osd_facecount(int opinput)
int         osd_firstpatch(int opinput, int face_id)
int         osd_firstpatch(string geometry, int face_id)
int         osd_limitsurface(int opinput, string attribute_name, int face_id, float u, float v, float & result)
int         osd_limitsurface(string geometry, string attribute_name, int face_id, float u, float v, float & result)
int         osd_limitsurfacevertex(int opinput, string attribute_name, int face_id, float u, float v, float & result)
int         osd_limitsurfacevertex(string geometry, string attribute_name, int face_id, float u, float v, float & result)
int         osd_patchcount(int opinput)
int         osd_patchcount(int opinput, int face_id)
int[]       osd_patches(int opinput, int face_id)
matrix3     outerproduct(vector vec1, vector vec2)
matrix2     outerproduct(vector2 vec1, vector2 vec2)
matrix      outerproduct(vector4 vec1, vector4 vec2)
vector      ow_nspace(vector n)
vector      ow_space(vector v)
vector      ow_vspace(vector v)
float       pack_inttosafefloat(int value)
int         pcexport(int handle, string channel_name, float value, ...)
int         pcexport(int handle, string channel_name, vector value, float radius, ...)
int         pcwrite(string filename, ...)
float       pnoise(vector pos, vector period)
float       point(int opinput, string attribute_name, int pointnumber)
float       point(string geometry, string attribute_name, int pointnumber)
float       pointattrib(int opinput, string attribute_name, int pointnumber, int & success)
float       pointattrib(string geometry, string attribute_name, int pointnumber, int & success)
int         pointattribsize(int opinput, string attribute_name)
int         pointattribtype(int opinput, string attribute_name)
int         pointvertex(int opinput, int pointindex)
float       pop(float[] & array)
int         pop(float[] & array, int index)
float       pow(float value, float power)
float       prim(int opinput, string attribute_name, int prim)
float       prim(string geometry, string attribute_name, int prim)
int         prim_attribute(string filename, float & adata, string att, int pn, vector uv)
float       primarclen(string filename, vector2 uv0, vector2 uv1, int pn)
float       primattrib(int opinput, string attribute_name, int prim, int & success)
float       primattrib(string geometry, string attribute_name, int prim, int & success)
int         primattribsize(int opinput, string attribute_name)
int         primattribtype(int opinput, string attribute_name)
float       primintrinsic(int opinput, string intrinsic, int prim)
float       primintrinsic(string geometry, string intrinsic, int prim)
int         primvertexcount(int opinput, int primindex)
vector      ptransform(vector p, matrix transform_matrix)
vector4     ptransform(vector4 p, matrix transform_matrix)
void        push(float[] & array, float value)
void        push(float[] & array, float[] values)
float       radians(float degrees)
void        rawbumpmap(string filename, float & du, float & dv, float u, float v)
void        rawbumpmap(string filename, float & du, float & dv, vector uvw)
void        rawbumpmapA(string filename, float & du, float & dv, float u, float v)
void        rawbumpmapA(string filename, float & du, float & dv, vector uvw)
void        rawbumpmapB(string filename, float & du, float & dv, float u, float v)
void        rawbumpmapB(string filename, float & du, float & dv, vector uvw)
void        rawbumpmapG(string filename, float & du, float & dv, float u, float v)
void        rawbumpmapG(string filename, float & du, float & dv, vector uvw)
void        rawbumpmapL(string filename, float & du, float & dv, float u, float v)
void        rawbumpmapL(string filename, float & du, float & dv, vector uvw)
void        rawbumpmapR(string filename, float & du, float & dv, float u, float v)
void        rawbumpmapR(string filename, float & du, float & dv, vector uvw)
vector      rawcolormap(string filename, float u, float v, ...)
vector4     rawcolormap(string filename, float u, float v, ...)
vector      rawcolormap(string filename, float u0, float v0, float u1, float v1, float u2, float v2, float u3, float v3, int samples, ...)
vector4     rawcolormap(string filename, float u0, float v0, float u1, float v1, float u2, float v2, float u3, float v3, int samples, ...)
vector      rawcolormap(string filename, vector uv, vector du, vector dv, int samples, ...)
vector4     rawcolormap(string filename, vector uv, vector du, vector dv, int samples, ...)
vector      rawcolormap(string filename, vector uv0, vector uv1, vector uv2, vector uv3, ...)
vector4     rawcolormap(string filename, vector uv0, vector uv1, vector uv2, vector uv3, ...)
vector      rawcolormap(string filename, vector uv0, vector uv1, vector uv2, vector uv3, int samples, ...)
vector4     rawcolormap(string filename, vector uv0, vector uv1, vector uv2, vector uv3, int samples, ...)
vector      rawcolormap(string filename, vector uvw, ...)
vector4     rawcolormap(string filename, vector uvw, ...)
vector      relbbox(int opinput, string primgroup, vector position)
vector      relpointbbox(int opinput, string pointgroup, vector position)
float       removeindex(float[] & array, int index)
int         removevalue(float[] & array, float value)
int         renderstate(material mat, string query, float & value)
void        resize(float[] & array, int size)
float       rint(float value)
void        rotate(matrix & mat, float amount, vector axis)
void        rotate(matrix2 & mat, float amount)
void        sample_bsdf(bsdf b, vector viewer, vector undocumented, int undocumented, vector & out_vector, vector & eval, float & pdf, int & type, float sx, float sy, int mask, ...)
void        sample_bsdf(bsdf b, vector viewer, vector & out_vector, vector & eval, float & pdf, int & type, float sx, float sy, int mask, ...)
void        sample_bsdf(bsdf b, vector viewer, vector & out_vector, vector & eval, float & pdf, int & type, float sx, float sy, ...)
void        sample_bsdf(bsdf b, vector viewer, vector & out_vector, vector & eval, int & type, float sx, float sy, int mask, ...)
void        sample_bsdf(bsdf b, vector viewer, vector & out_vector, vector & eval, int & type, float sx, float sy, ...)
vector2     sample_cauchy(float scale, vector2 u)
void        sample_cdf(float cdf[], float u, int &index, float &x, float &pdf)
int         sample_cdf(float cdf[], float u)
void        scale(matrix & mat, vector scale_vector)
void        scale(matrix2 & mat, vector2 scale_vector)
float       select(int conditional, float a, float b)
float[]     serialize(vector[] array)
vector4     set(float rowx, float rowy, float rowz, float roww)
vector2     set(float u, float v)
float       set(float value)
vector      set(float x, float y, float z)
matrix      set(float xx, float xy, float xz, float xw, float yx, float yy, float yz, float yw, float zx, float zy, float zz, float zw, float wx, float wy, float wz, float ww)
matrix3     set(float xx, float xy, float xz, float yx, float yy, float yz, float zx, float zy, float zz)
matrix2     set(float xx, float xy, float yx, float yy)
matrix3     set(vector rowx, vector rowy, vector rowz)
matrix2     set(vector2 rowu, vector2 rowv)
matrix      set(vector4 rowx, vector4 rowy, vector4 rowz, vector4 roww)
int         setattrib(int geohandle, string attribclass, string name, int elemnum, int vtxofprim, float value)
int         setdetailattrib(int geohandle, string name, float value)
int         setpointattrib(int geohandle, string name, int pt, float value)
int         setpointgroup(int geohandle, string name, int pt, int value)
int         setprimattrib(int geohandle, string name, int prim, float value)
int         setprimgroup(int geohandle, string name, int prim, int value)
int         setprimintrinsic(int geohandle, string name, int prim, float value)
int         setvertexattrib(int geohandle, string name, int elemnum, int vtxofprim, float value)
int         setvertexgroup(int geohandle, string name, int prim, int vtx, int value)
float       shadowmap(string filename, vector rect1, vector rect2, vector rect3, vector rect4, float spread, float bias, float quality)
int         shimport(string name, float & value)
float       sign(float value)
float       solid_angle(bsdf b, int mask, ...)
bsdf[]      split(bsdf bsdf_undocumented)
void        split_bsdf(bsdf[] lobes, bsdf b, float[] &weights)
void        split_bsdf(bsdf[] lobes, bsdf b, float[] &weights, int mask)
void        split_bsdf(bsdf[] lobes, bsdf b, float[] &weights, int mask, int type)
void        split_bsdf(bsdf[] lobes, bsdf b, float[] &weights, int mask, int type, float u)
void        split_bsdf(bsdf[] lobes, bsdf b, float[] &weights, int mask, int type, float u, float[] cdf)
bsdf[]      split_bsdf(bsdf b, float[] &weights)
bsdf[]      split_bsdf(bsdf b, float[] &weights, int mask, int type)
bsdf[]      split_bsdf(bsdf b, float[] &weights, int mask, int type, float u)
bsdf[]      split_bsdf(bsdf b, float[] &weights, int mask, int type, float u, float[] cdf)
void        storelightexport(string lightname, string exportname, float value)
vector      swizzle(vector xyz, int i0, int i1, int i2)
vector2     swizzle(vector2 uv, int i0, int i1)
vector4     swizzle(vector4 xyzw, int i0, int i1, int i2, int i3)
int         teximport(string map, string attribute, float & value)
string      texprintf(float u, float v, string format, ...)
void        texture3dBox(string filename, string channel, vector & min, vector & max, ...)
vector      toNDC(vector point)
void        translate(matrix mat&, vector amount)
matrix      transpose(matrix mat)
float       trunc(float value)
vector      tw_nspace(vector n)
vector      tw_space(vector v)
vector      tw_vspace(vector v)
int         unpack_intfromsafefloat(float value)
vector[]    unserialize(float[] array)
void        upush(float[] & array, float value)
int         uvunwrap(string undocumented, float u, float v, float time, vector &P, vector &I) 
int         uvunwrap(string undocumented, float u, float v, float time, vector &undocumented, vector &undocumented, vector &undocumented, vector &undocumented) 
float       vertex(int opinput, string attribute_name, int linearvertex)
float       vertex(int opinput, string attribute_name, int prim, int vertex)
float       vertex(string geometry, string attribute_name, int linearvertex)
float       vertex(string geometry, string attribute_name, int prim, int vertex)
float       vertexattrib(int opinput, string attribute_name, int linearvertex, int & success)
float       vertexattrib(string geometry, string attribute_name, int linearvertex, int & success)
int         vertexattribsize(int opinput, string attribute_name)
int         vertexattribtype(int opinput, string attribute_name)
int         vertexnext(int opinput, int linearvertex)
int         vertexprev(int opinput, int linearvertex)
int         vertexprim(int opinput, int linearvertex)
int         vertexprimindex(int opinput, int linearvertexnum)
void        vnoise(float position, float jitter, int & seed, float & f1, float & f2, float & pos1, float & pos2)
void        vnoise(float position, float jitter, int & seed, float & f1, float & f2, float & pos1, float & pos2, int period)
void        vnoise(float position_s, float position_t, float jitter_s, float jitter_t, int & seed, float & f1, float & f2, float & pos1_u, float & pos1_v, float & pos2_u, float & pos2_v)
void        vnoise(float position_s, float position_t, float jitter_s, float jitter_t, int & seed, float & f1, float & f2, float & pos1_u, float & pos1_v, float & pos2_u, float & pos2_v, int period_s, int period_t)
void        vnoise(vector position, vector jitter, int & seed, float & f1, float & f2, float & pos1, float & pos2)
void        vnoise(vector position, vector jitter, int & seed, float & f1, float & f2, float & pos1, float & pos2, int period_x, int period_y, int period_z)
void        vnoise(vector position, vector jitter, int & seed, float & f1, float & f2, float & pos1, float & pos2, vector period)
void        vnoise(vector position, vector jitter, int & seed, float & f1, float & f2, vector & pos1, vector & pos1)
void        vnoise(vector position, vector jitter, int & seed, float & f1, float & f2, vector & pos1, vector & pos1, int period_x, int period_y, int period_z)
void        vnoise(vector position, vector jitter, int & seed, float & f1, float & f2, vector & pos1, vector & pos1, vector period)
void        vnoise(vector4 position, vector4 jitter, int & seed, float & f1, float & f2, vector4 & pos1, vector4 & pos2)
void        vnoise(vector4 position, vector4 jitter, int & seed, float & f1, float & f2, vector4 & pos1, vector4 & pos2, int period_x, int period_y, int period_z, int period_w)
void        vnoise(vector4 position, vector4 jitter, int & seed, float & f1, float & f2, vector4 & pos1, vector4 & pos2, vector4 period)
vector      vtransform(string fromspace, string tospace, vector v)
vector      vtransform(string tospace, vector v)
vector      vtransform(vector v, matrix transform_matrix)
void        wnoise(float position, int & seed, float & f1, float & f2)
void        wnoise(float position, int & seed, float & f1, float & f2, float & f3, float & f4)
void        wnoise(float position, int & seed, float & f1, float & f2, float & f3, float & f4, int period)
void        wnoise(float position, int & seed, float & f1, float & f2, int period)
void        wnoise(float position_s, float position_t, int & seed, float & f1, float & f2)
void        wnoise(float position_s, float position_t, int & seed, float & f1, float & f2, float & f3, float & f4)
void        wnoise(float position_s, float position_t, int & seed, float & f1, float & f2, float & f3, float & f4, int period_s, int period_v)
void        wnoise(float position_s, float position_t, int & seed, float & f1, float & f2, int period_s, int period_v)
void        wnoise(vector position, int & seed, float & f1, float & f2)
void        wnoise(vector position, int & seed, float & f1, float & f2, float & f3, float & f4)
void        wnoise(vector position, int & seed, float & f1, float & f2, float & f3, float & f4, int period_x, int period_y, int period_z)
void        wnoise(vector position, int & seed, float & f1, float & f2, int period_x, int period_y, int period_z)
void        wnoise(vector4 position, int & seed, float & f1, float & f2)
void        wnoise(vector4 position, int & seed, float & f1, float & f2, float & f3, float & f4)
void        wnoise(vector4 position, int & seed, float & f1, float & f2, float & f3, float & f4, int period_x, int period_y, int period_z, int period_w)
void        wnoise(vector4 position, int & seed, float & f1, float & f2, int period_x, int period_y, int period_z, int period_w)
vector      wo_nspace(vector n)
vector      wo_space(vector v)
vector      wo_vspace(vector v)
vector      wt_nspace(vector n)
vector      wt_space(vector v)
vector      wt_vspace(vector v)
float       xyzdist(int opinput, string primgroup, vector pt)
float       xyzdist(int opinput, string primgroup, vector pt, int & prim, vector & uv)
float       xyzdist(int opinput, string primgroup, vector pt, int & prim, vector & uv, float maxdist)
float       xyzdist(int opinput, vector pt)
float       xyzdist(int opinput, vector pt, int & prim, vector & uv)
float       xyzdist(int opinput, vector pt, int & prim, vector & uv, float maxdist)'''


HFS = 'C:/Program Files/Side Effects Software/Houdini 16.0.600'
assert(op.exists(HFS))

# Open vex.zip containing various helpcards including functions.
# Open all function helpcards and find all function usage examples.
docsigs = set()
with zipfile.ZipFile(op.join(HFS, 'houdini/help/vex.zip')) as z:
    for path in z.namelist():
        if op.dirname(path) == 'functions':
            with z.open(path) as f:
                docsigs.update(re.findall(r'^(?:\*|:usage:) `(.*)`$', f.read().decode(), re.M))

parsed = parse_sigs(docsigs)

# Parse overrides on top of documentation signatures.
for f, sigs in parse_sigs(overrides.split('\n')).items():
    # Function not exists.
    if f not in parsed:
        parsed[f] = sigs
        continue

    # Function exists. Search for unique signatures.
    for over_s in sigs['sigs']:
        for s in copy.deepcopy(parsed[f]['sigs']):
            # Non-unique signature found. Override argument names.
            if s['args'] == over_s['args']:
                parsed[f]['sigs'].remove(s)
                parsed[f]['sigs'].insert(0, over_s)
                break  # Suppress for-else clause.

        # Non-unique signatures not found (= the signature is unique).
        else:
            parsed[f]['sigs'].insert(0, over_s)

def test():
    f = random.choice(list(parsed.keys()))
    print(
        '\nSignatures parsed: {}'.format(sum(len(parsed[f]['sigs']) for f in parsed)),
        '\nExample function:', f
    )
    pprint(parsed[f])

test()


Signatures parsed: 1886 
Example function: agentclipsample
{'sigs': [{'argnames': ['opinput', 'prim', 'clipname', 'time', 'channel_index'],
           'args': ['const int',
                    'const int',
                    'const string',
                    'const float',
                    'const int'],
           'return': 'float'},
          {'argnames': ['geometry', 'prim', 'clipname', 'time', 'channel'],
           'args': ['const string',
                    'const int',
                    'const string',
                    'const float',
                    'const string'],
           'return': 'float'},
          {'argnames': ['opinput', 'prim', 'clipname', 'time', 'channel'],
           'args': ['const int',
                    'const int',
                    'const string',
                    'const float',
                    'const string'],
           'return': 'float'},
          {'argnames': ['geometry',
                        'prim',
                        '

### Fill function signatures with argument names

Since documentation is describing about half of the function signatures,
rest will be filled from current data by simple constitution rules with
aiding from overrides file.

Iterate over `vcc` and find similar signatures in `parsed`.
Exact matches covers about one third of all existing signatures.
Other signatures filled by trivial "guessing" operations:

1. Return type changes:
```
    Docs:   int foo(string bar)
    VCC:    float foo(string)
    Result: foo(string bar)
```

2. An array functions with type and two arguments changing and limited names scope.
```
    Docs:   void foo(int value, int[] array)
    VCC:    void foo(float, float[])
    Result: float foo(float value, float[] array)
```

3. Of the arguments type changes.
```
    Docs:   string foo(int bar)
    VCC:    string foo(float)
    Result: string foo(float bar)
```

4. Return and argument types changes.
```
    Docs:   int foo(int bar)
    VCC:    float foo(float)
    Result: float foo(float bar)
```

Although a lot more trivial guesses may be done, it just not worth to do it to
get 5-10 extra functions fully defined. Better to define them in overrides,
which already contains couple of hundred non-trivial function signatures.

#### Full data from filling
```yaml
foo:
  sigs:
  - argnames:
    - bar
    fillstate: EXACT
```

In [10]:
filled = copy.deepcopy(vcc)

for f in filled:
    for sig in filled[f]['sigs']:
        def simplify(t):
            '''Simplify real type names to make comparisons more robust.'''
            return t.replace('const ', '').replace('export ', '').strip()

        def fill():
            '''Wrapped into a function to use return.'''
            # No arguments. Nothing to fill.
            if not sig['args']:
                return 'NOARG', []

            # Need to update functions.vfl.
            if f not in parsed:
                return 'NODOC', []

            # Shortcuts.
            rtrn = sig['return']
            args = [simplify(s) for s in sig['args']]

            # Exact match.
            for psig in parsed[f]['sigs']:
                pargs = [simplify(a) for a in psig['args']]
                if args == pargs and rtrn == psig['return']:
                    return 'EXACT', psig['argnames'][:]

            # Guess for an array functions with type and two arguments changing.
            for psig in parsed[f]['sigs']:
                pargs = [simplify(a) for a in psig['args']]
                scope = ['append', 'find', 'getcomp', 'insert', 'pop',
                         'push', 'removeindex', 'removevalue', 'reorder',
                         'reverse', 'select', 'setcomp', 'slice', 'upush']
                if f in scope and len(args) == len(pargs):
                    if sum(1 for x, y in zip(args, pargs) if x != y) == 2 or \
                            sum(1 for x, y in zip(args, pargs) if x != y) == 1 and rtrn != psig['return']:
                        return 'ARRAY', psig['argnames'][:]

            # Guess for return type changes.
            for psig in parsed[f]['sigs']:
                pargs = [simplify(a) for a in psig['args']]
                if args == pargs:
                    return 'RETRN', psig['argnames'][:]

            # Guess for one of the arguments type changes.
            for psig in parsed[f]['sigs']:
                pargs = [simplify(a) for a in psig['args']]
                if len(args) == len(pargs) and rtrn == psig['return']:
                    if sum(1 for x, y in zip(args, pargs) if x != y) == 1:
                        return 'ARGMT', psig['argnames'][:]

            # Guess for a combination of return and argument types changes.
            for psig in parsed[f]['sigs']:
                pargs = [simplify(a) for a in psig['args']]
                if len(args) == len(pargs):
                    if sum(1 for x, y in zip(args, pargs) if x != y) == 1:
                        return 'REARG', psig['argnames'][:]

            # Can't guess. Should update functions.vfl.
            return 'ERROR', []

        sig['fillstate'], sig['argnames'] = fill()


# Print debug info about filled signatures.
for f in sorted(filled):
    for sig in filled[f]['sigs']:
        def logsig(symbol):
            '''Print function info in a console.'''
            def cleanup(raw_arg):
                '''Cleanup visual representation of an argument.'''
                arg = simplify(raw_arg)
                if 'const' not in raw_arg:
                    arg += ' &'
                return arg

            args = zip([cleanup(a) for a in sig['args']], sig['argnames'])
            args = ', '.join([' '.join(pair) for pair in args])
            if 'variadic' in sig:
                args += ', ...' if args else '...'

            sigstr = '{} {} {:<10}{}({})'.format(symbol, sig['fillstate'], sig['return'], f, args)
            print(sigstr)

        if sig['fillstate'] == 'ERROR':
            print(sig)
            raise Exception('Unfilled f: "%s".' %f)
        elif sig['fillstate'] == 'NODOC':
            raise Exception('Unknown f: "%s". Update overrides.' %f)
        elif sig['fillstate'] in 'EXACT':
#             logsig('+')
            pass
        elif sig['fillstate'] in 'NOARG':
#             logsig('+')
            pass
        elif sig['fillstate'] == 'RETRN':
#             logsig('?')
            pass
        elif sig['fillstate'] == 'ARGMT':
#             logsig('?')
            pass
        elif sig['fillstate'] == 'REARG':
#             logsig('?')
            pass
        elif sig['fillstate'] == 'ARRAY':
#             logsig('?')
            pass
        else:
            raise Exception('Memory error: something forgotten.')

# Discard redundant entries to be more DRY.
for f, sigs in filled.items():
    filled[f] = {
        'sigs': [{'argnames': s['argnames'][:], 'fillstate': s['fillstate']} for s in sigs['sigs']]
    }

### Parse online docs and find earliest Houdini version where the function was introduced

#### Resulting data
```yaml
foo:
  added: '14.0'
```

In [11]:
online = {}

r = requests.get('http://www.sidefx.com/docs/houdini16.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['16.0'] = {a.text for a in s.find_all('a', class_='vex')}

r = requests.get('http://www.sidefx.com/docs/houdini15.5/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['15.5'] = {a.text for a in s.find_all('a', class_='vex')}

r = requests.get('http://www.sidefx.com/docs/houdini15.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['15.0'] = {a.text for a in s.find(class_='hitlist').find_all('a')}

r = requests.get('http://www.sidefx.com/docs/houdini14.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['14.0'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini13.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['13.0'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini12.5/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['12.5'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini12.1/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['12.1'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini12.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['12.0'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini11.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['11.0'] = {td.find('a').text for td in s.find_all('td', class_='linktitlecell')}

r = requests.get('http://www.sidefx.com/docs/houdini10.0/vex/functions/_index')
s = bs4.BeautifulSoup(r.content, 'html.parser')
online['10.0'] = {a.text for a in s.find_all('a', class_='iconlink')}

# Rebuild dictionary so only unique function with earliest possible version left.
history = {
    '16.0' : sorted(online['16.0'] - online['15.5']),
    '15.5' : sorted((online['15.5'] - online['15.0']) & online['16.0']),
    '15.0' : sorted((online['15.0'] - online['14.0']) & online['16.0']),
    '14.0' : sorted((online['14.0'] - online['13.0']) & online['16.0']),
    '13.0' : sorted((online['13.0'] - online['12.5']) & online['16.0']),
    '12.5' : sorted((online['12.5'] - online['12.1']) & online['16.0']),
    '12.1' : sorted((online['12.1'] - online['12.0']) & online['16.0']),
    '12.0' : sorted((online['12.0'] - online['11.0']) & online['16.0']),
    '11.0' : sorted((online['11.0'] - online['10.0']) & online['16.0'])
}
history = {f: {'added': v} for v, funs in history.items() for f in funs}

def test():
    f = random.choice(list(history.keys()))
    print(
        'Total history records: %d' % len(history),
        '\nRandom record: %s' % f,
    )
    pprint(history[f])

test()

Total history records: 449 
Random record: uvsample
{'added': '16.0'}


### Generate simplified HTML pages from data available in Houdini help documentation

#### Resulting data
```yaml
foo:
  help: <html>foo</html>
```

In [12]:
# Page retrieving will take 15 minutes to complete if not cached.
index = 'http://localhost:48626/vex/functions/'

def ret(url):
    return requests.get(url).content

index_soup = bs4.BeautifulSoup(ret(index), 'html.parser')

if op.exists('functions.p'):
    with open('functions.p', 'rb') as f:
        pages = pickle.load(f)

else:
    available_pages = {a.text for a in index_soup.find_all('a', class_='vex')}
    pages = {f : None
             for f in sorted(available_pages)
             if f in vcc}
    for i, f in enumerate(pages):
        pages[f] = ret(index + f)
        if i>0 and (i%25==0 or i==len(pages)-1):
            print('Loaded %d pages.' % i)

    with open('functions.p', 'wb') as f:
        pickle.dump(pages, f)

Loaded 25 pages.
Loaded 50 pages.
Loaded 75 pages.
Loaded 100 pages.
Loaded 125 pages.
Loaded 150 pages.
Loaded 175 pages.
Loaded 200 pages.
Loaded 225 pages.
Loaded 250 pages.
Loaded 275 pages.
Loaded 300 pages.
Loaded 325 pages.
Loaded 350 pages.
Loaded 375 pages.
Loaded 400 pages.
Loaded 425 pages.
Loaded 450 pages.
Loaded 475 pages.
Loaded 500 pages.
Loaded 525 pages.
Loaded 550 pages.
Loaded 575 pages.
Loaded 600 pages.
Loaded 625 pages.
Loaded 650 pages.
Loaded 675 pages.
Loaded 700 pages.
Loaded 701 pages.


In [13]:
def typesort(t):
    '''Return positions of type in types array.'''
    order = [
        'void', 'int', 'string', 'float', 'vector2', 'vector', 'vector4',
        'matrix2', 'matrix3', 'matrix', 'light', 'material', 'bsdf',
        'int[]', 'string[]', 'float[]', 'vector2[]', 'vector[]', 'vector4[]',
        'matrix2[]', 'matrix3[]', 'matrix[]', 'bsdf[]', 'light[]', 'material[]'
    ]
    return order.index(t.replace('const', '')
                        .replace('export', '')
                        .replace('&', '')
                        .strip())


def all_signatures(f):
    '''Return list of human-readable signature strings.'''
    global vcc, filled

    sortbyme = []
    sigs = []
    for vccsig, docsig in zip(vcc[f]['sigs'],
                              filled[f]['sigs']):
        sigargs = []
        for type_, name in zip(vccsig['args'], docsig['argnames']):
            if 'const' not in type_:
                name = '&' + name
            type_ = type_.replace('const ', '').replace('export ', '')
            sigargs.append('%s %s' %(type_, name))

        if 'variadic' in vccsig:
            sigargs.append('...')

        sig = '{} {}({})'.format(vccsig['return'], f, ', '.join(sigargs))
       
        all_contexts = {'cvex', 'light', 'surface',
                        'pop', 'chop', 'shadow',
                        'cop2', 'image3d',
                        'displace', 'sop', 'fog'}
        shading_contexts = {'light', 'surface',
                            'shadow', 'displace', 'fog'}
        contexts = vccsig['contexts']
        if set(contexts) == all_contexts:
            contexts = []
        elif set(contexts) == shading_contexts:
            contexts = ['shading contexts']
        
        sortbyme.append(tuple(typesort(t) for t in [vccsig['return'], *vccsig['args']]))
        sigs.append((sig, contexts))

    _, sigs = zip(*sorted(zip(sortbyme, sigs)))
    return sigs


def tags(f):
    '''Return tags from function index page.'''
    global index_soup
    try:
        return index_soup.find('li', **{'data-title': f})['data-tags'].split()
    except (TypeError, KeyError):
        return []


def make_helpcard(f):
    global vcc, args, history, pages
    index = 'https://www.sidefx.com/docs/houdini/vex/functions/'
    s = bs4.BeautifulSoup(pages[f], 'html.parser')

    nonmeta, = s.main.select('#content')
    nonmeta['id'] = 'helpcard'

    # Strip redundant tags which does not work good inside ST3 popups.
    for span in nonmeta.find_all('span', class_='line'):
        span.wrap(s.new_tag('code', **{'class': 'line'}))
        span.unwrap()
    for var in nonmeta.find_all('var'):
        var.wrap(s.new_tag('code'))
        var.unwrap()
    for div in nonmeta.find_all('div', class_='def'):
        div.wrap(s.new_tag('p'))
        div.unwrap()
    for code in nonmeta.find_all('code', class_='codehilite'):
        code.code.unwrap()
    for div in nonmeta.find_all('div', class_='clear'):
        div.unwrap()
    for p in nonmeta.find_all('p', class_='label'):
        p.unwrap()
    for tag in nonmeta.find_all(['pre', 'dl', 'span', 'section']):
        tag.unwrap()
        
    # Append related section.
    related = s.select('#postmeta')[0].select('a.Vex')
    if related:
        h = s.new_tag('h2')
        h.string = 'See also'
        nonmeta.append(h)

        for a in related:
            title = a.get('title')
            text = a.text
            a = s.new_tag('a', href=index+a['href'])
            a.string = text

            p = s.new_tag('p')
            p.append(a)

            if title:
                div = s.new_tag('div', **{'class': 'related-summary'})
                div.string = title
                p.append(div)

            nonmeta.append(p)

    for table in nonmeta.find_all('table'):
        table.decompose()

    # Destroy default documentation signatures.
    # TODO: Remove small indents in ST3 caused by properly formatted HTML code by removing '\n' strings.
    for c in nonmeta.children:
        if c != '\n':
            if (c.name == 'ul' and 'bullets' in c['class']) \
                    or (c.name == 'div' and 'usage_group' in c['class']):
                c.decompose()
            break

    # Make URLs full.
    for a in nonmeta.find_all('a'):
        a['href'] = index + a['href']

    # Images from web still not work for me in ST3. Replace with links.
    for img in nonmeta.find_all('img'):
        img_link = s.new_tag('a', href=index + img['src'])
        img_link.string = '[Image \U0001f517]'
        img.wrap(img_link)
        img.unwrap()

    # Fix not working <pre> tag with non-breaking spaces.
    for code in nonmeta.find_all('code', class_='line'):
        newtext = code.text.lstrip(' ')
        code.string = '\xa0' * (len(code.text)-len(newtext)) + newtext

    # Add summary paragraph.
    summary = s.main.select('.summary')
    if not summary:
        summary = s.new_tag('p', **{'class': 'summary'})
    else:
        summary = summary[0]
    nonmeta.insert(0, summary)

    # Insert title and online documentation hyperlink.
    title = s.new_tag('a', href=index+f)
    title.string = f
    heading = s.new_tag('h1')
    heading.append(title)
    summary.insert_before(heading)
    
    # Insert generated signatures.
    ul = s.new_tag('ul')
    for sig, contexts in all_signatures(f):
        li = s.new_tag('li')
        code = s.new_tag('code')
        code.string = sig
        li.append(code)
        for c in contexts:
            span = s.new_tag('span', **{'class': 'pillow'})
            span.string = c
            li.append(span)
            span.wrap(s.new_tag('span', **{'class': 'padder'}))
        ul.append(li)
    summary.insert_after(ul)

    # Insert tags
    ftags = tags(f)
    if f in history:
        ftags.append(history[f]['added'])
    if ftags:
        outspan = s.new_tag('span')
        for ftag in sorted(ftags):
            span = s.new_tag('span', **{'class': 'pillow'})
            span.string = ftag
            outspan.append(span)
            span.wrap(s.new_tag('span', **{'class': 'padder'}))
        summary.insert_before(outspan)
   
    # print(nonmeta.prettify())
    return str(nonmeta)

# # For quick testing.
# with open('../commands/helpcards.json', 'w') as f:
#     helpcards = {
#         'vex': {
#             'point' : make_helpcard('point'),
#             'sample_bsdf' : make_helpcard('sample_bsdf'),
#             'ashikhmin' : make_helpcard('ashikhmin'),
#             'sample_cdf' : make_helpcard('sample_cdf'),
#             'pcopen' : make_helpcard('pcopen'),
#             'pcwrite' : make_helpcard('pcwrite'),
#             'addprim' : make_helpcard('addprim'),
#         }
#     }
#     json.dump(helpcards, f, indent=4, sort_keys=True)

helpcards = {}
for i, f in enumerate(sorted(pages)):
    try:
        helpcard = make_helpcard(f)
        helpcards[f] = {'help': helpcard}
    except:
        print(f)
        raise
    if i>0 and (i%25==0 or i==len(pages)-1):
        print('Finished %d helpcards.' % i)

Finished 25 helpcards.
Finished 50 helpcards.
Finished 75 helpcards.
Finished 100 helpcards.
Finished 125 helpcards.
Finished 150 helpcards.
Finished 175 helpcards.
Finished 200 helpcards.
Finished 225 helpcards.
Finished 250 helpcards.
Finished 275 helpcards.
Finished 300 helpcards.
Finished 325 helpcards.
Finished 350 helpcards.
Finished 375 helpcards.
Finished 400 helpcards.
Finished 425 helpcards.
Finished 450 helpcards.
Finished 475 helpcards.
Finished 500 helpcards.
Finished 525 helpcards.
Finished 550 helpcards.
Finished 575 helpcards.
Finished 600 helpcards.
Finished 625 helpcards.
Finished 650 helpcards.
Finished 675 helpcards.
Finished 700 helpcards.
Finished 701 helpcards.


### Merge everything into mega JSON

Previously generated structures:

`vcc`
```yaml
foo:
  sigs:
  - args:
    - int
    return:
    - float
    contexts:
    - cvex
    variadic: true
    variadic_pair: true
    external: true
```

`filled`
```yaml
foo:
  sigs:
  - argnames:
    - bar
    fillstate: EXACT
```

`history`
```yaml
foo:
  added: '14.0'
```

`helpcards`
```yaml
foo:
  help: <html>foo</html>
```

#### Goal structure
`functions`
```yaml
foo:
  added: '14.0'
  help: <html>foo</html>
  sigs:
  - args:
    - int
    argnames:
    - bar
    return:
    - float
    contexts:
    - cvex
    variadic: true
    variadic_pair: true
    external: true
    fillstate: EXACT
```

In [14]:
functions = {f: {} for f in vcc}

for f in functions:
    if f in history:
        functions[f]['added'] = history[f]['added']
    if f in helpcards: 
        functions[f]['help'] = helpcards[f]['help']

    functions[f]['sigs'] = []
    for vs, fs in zip(vcc[f]['sigs'], filled[f]['sigs']):
        s = {
            'args': vs['args'],
            'return': vs['return'],
            'contexts': vs['contexts'][:],
            'argnames': fs['argnames'],
            'fillstate': fs['fillstate'],
        }
        if 'variadic' in vs:
            s['variadic'] = vs['variadic']
        if 'variadic_pair' in vs:
            s['variadic_pair'] = vs['variadic_pair']

        functions[f]['sigs'].append(s)


with open('functions.json', 'w') as f:
    json.dump(functions, f, indent=4, sort_keys=True)


def test():
    f = random.choice(list(set(vcc) & set(filled) & set(history) & set(helpcards)))
    print('Lengths: vcc:{} filled:{} history:{} helpcards:{}'.format(len(vcc),
                                                                     len(filled),
                                                                     len(history),
                                                                     len(helpcards)))
    print('\nRandom function: %s' %f)
    print('\n[vcc]:')
    pprint(vcc[f])
    print('\n\n[filled]:')
    pprint(filled[f])
    print('\n\n[history]:')
    pprint(history[f])
    print('\n\n[helpcards]:')
    pprint(helpcards[f])
    print('\n\nHelp HTML:')
    print(helpcards[f]['help'])
    print('\n\nFinal structure:')
    pprint(functions[f])
    return (HTML(helpcards[f]['help']))

test()

Lengths: vcc:720 filled:720 history:449 helpcards:702

Random function: hasprimattrib

[vcc]:
{'sigs': [{'args': ['const int', 'const string'],
           'contexts': ['surface',
                        'displace',
                        'light',
                        'shadow',
                        'fog',
                        'chop',
                        'pop',
                        'sop',
                        'cop2',
                        'image3d',
                        'cvex'],
           'external': True,
           'return': 'int'},
          {'args': ['const string', 'const string'],
           'contexts': ['surface',
                        'displace',
                        'light',
                        'shadow',
                        'fog',
                        'chop',
                        'pop',
                        'sop',
                        'cop2',
                        'image3d',
                        'cvex'],
           'externa

In [15]:
# Make some history information to be visible in online Notebook viewers.
def diff(a, b):
    return sorted(online[b] - online[a])

def link(version, function):
    return ('[{function}](http://www.sidefx.com/docs/houdini{version}/vex/functions/{function})'.format(**locals()))

def block(new, old):
    removed = '\n'.join('-' + link(old, f) for f in diff(new, old))
    added = '\n'.join('+' + link(new, f) for f in diff(old, new))
    total = len(online[new])
    return '''
#### New in {new}

{removed}{added}

Total: {total} functions.
    '''.format(**locals())

markup = '### VEX History'
markup += block('16.0', '15.5')
markup += block('15.5', '15.0')
markup += block('15.0', '14.0')
markup += block('14.0', '13.0')
markup += block('13.0', '12.5')
markup += block('12.5', '12.1')
markup += block('12.1', '12.0')
markup += block('12.0', '11.0')
markup += block('11.0', '10.0')
markup += '''
#### From 10.0

{}

Total: {} functions.'''.format('\n'.join([link('10.0', f) for f in sorted(online['10.0'] & online['16.0'])]),
                               len(online['10.0'] & online['16.0']))

Markdown(markup)

### VEX History
#### New in 16.0

-[getuvobjects](http://www.sidefx.com/docs/houdini15.5/vex/functions/getuvobjects)+[agentcliptransformgroups](http://www.sidefx.com/docs/houdini16.0/vex/functions/agentcliptransformgroups)
+[agenttransformgroupmember](http://www.sidefx.com/docs/houdini16.0/vex/functions/agenttransformgroupmember)
+[agenttransformgroups](http://www.sidefx.com/docs/houdini16.0/vex/functions/agenttransformgroups)
+[chadd](http://www.sidefx.com/docs/houdini16.0/vex/functions/chadd)
+[chattr](http://www.sidefx.com/docs/houdini16.0/vex/functions/chattr)
+[chattrnames](http://www.sidefx.com/docs/houdini16.0/vex/functions/chattrnames)
+[chindex](http://www.sidefx.com/docs/houdini16.0/vex/functions/chindex)
+[chinputlimits](http://www.sidefx.com/docs/houdini16.0/vex/functions/chinputlimits)
+[chnames](http://www.sidefx.com/docs/houdini16.0/vex/functions/chnames)
+[chreadbuf](http://www.sidefx.com/docs/houdini16.0/vex/functions/chreadbuf)
+[chremove](http://www.sidefx.com/docs/houdini16.0/vex/functions/chremove)
+[chremoveattr](http://www.sidefx.com/docs/houdini16.0/vex/functions/chremoveattr)
+[chrename](http://www.sidefx.com/docs/houdini16.0/vex/functions/chrename)
+[chresizebuf](http://www.sidefx.com/docs/houdini16.0/vex/functions/chresizebuf)
+[chsetattr](http://www.sidefx.com/docs/houdini16.0/vex/functions/chsetattr)
+[chsetlength](http://www.sidefx.com/docs/houdini16.0/vex/functions/chsetlength)
+[chsetrate](http://www.sidefx.com/docs/houdini16.0/vex/functions/chsetrate)
+[chsetstart](http://www.sidefx.com/docs/houdini16.0/vex/functions/chsetstart)
+[chwritebuf](http://www.sidefx.com/docs/houdini16.0/vex/functions/chwritebuf)
+[create_cdf](http://www.sidefx.com/docs/houdini16.0/vex/functions/create_cdf)
+[create_pdf](http://www.sidefx.com/docs/houdini16.0/vex/functions/create_pdf)
+[efit](http://www.sidefx.com/docs/houdini16.0/vex/functions/efit)
+[expandedgegroup](http://www.sidefx.com/docs/houdini16.0/vex/functions/expandedgegroup)
+[geounwrap](http://www.sidefx.com/docs/houdini16.0/vex/functions/geounwrap)
+[getbbox_center](http://www.sidefx.com/docs/houdini16.0/vex/functions/getbbox_center)
+[getbbox_max](http://www.sidefx.com/docs/houdini16.0/vex/functions/getbbox_max)
+[getbbox_min](http://www.sidefx.com/docs/houdini16.0/vex/functions/getbbox_min)
+[getbbox_size](http://www.sidefx.com/docs/houdini16.0/vex/functions/getbbox_size)
+[getgroupid](http://www.sidefx.com/docs/houdini16.0/vex/functions/getgroupid)
+[getlocalcurvature](http://www.sidefx.com/docs/houdini16.0/vex/functions/getlocalcurvature)
+[getmaterialid](http://www.sidefx.com/docs/houdini16.0/vex/functions/getmaterialid)
+[getobjectid](http://www.sidefx.com/docs/houdini16.0/vex/functions/getobjectid)
+[getpointbbox_center](http://www.sidefx.com/docs/houdini16.0/vex/functions/getpointbbox_center)
+[getpointbbox_max](http://www.sidefx.com/docs/houdini16.0/vex/functions/getpointbbox_max)
+[getpointbbox_min](http://www.sidefx.com/docs/houdini16.0/vex/functions/getpointbbox_min)
+[getpointbbox_size](http://www.sidefx.com/docs/houdini16.0/vex/functions/getpointbbox_size)
+[hasmetadata](http://www.sidefx.com/docs/houdini16.0/vex/functions/hasmetadata)
+[inedgegroup](http://www.sidefx.com/docs/houdini16.0/vex/functions/inedgegroup)
+[lightbounces](http://www.sidefx.com/docs/houdini16.0/vex/functions/lightbounces)
+[metadata](http://www.sidefx.com/docs/houdini16.0/vex/functions/metadata)
+[nedgesgroup](http://www.sidefx.com/docs/houdini16.0/vex/functions/nedgesgroup)
+[ninputs](http://www.sidefx.com/docs/houdini16.0/vex/functions/ninputs)
+[ocean_sample](http://www.sidefx.com/docs/houdini16.0/vex/functions/ocean_sample)
+[ocio_import](http://www.sidefx.com/docs/houdini16.0/vex/functions/ocio_import)
+[ocio_spaces](http://www.sidefx.com/docs/houdini16.0/vex/functions/ocio_spaces)
+[ocio_transform](http://www.sidefx.com/docs/houdini16.0/vex/functions/ocio_transform)
+[opparentbonetransform](http://www.sidefx.com/docs/houdini16.0/vex/functions/opparentbonetransform)
+[opparenttransform](http://www.sidefx.com/docs/houdini16.0/vex/functions/opparenttransform)
+[opparmtransform](http://www.sidefx.com/docs/houdini16.0/vex/functions/opparmtransform)
+[oppreconstrainttransform](http://www.sidefx.com/docs/houdini16.0/vex/functions/oppreconstrainttransform)
+[oppretransform](http://www.sidefx.com/docs/houdini16.0/vex/functions/oppretransform)
+[osd_lookupface](http://www.sidefx.com/docs/houdini16.0/vex/functions/osd_lookupface)
+[osd_lookuppatch](http://www.sidefx.com/docs/houdini16.0/vex/functions/osd_lookuppatch)
+[planepointdistance](http://www.sidefx.com/docs/houdini16.0/vex/functions/planepointdistance)
+[planesphereintersect](http://www.sidefx.com/docs/houdini16.0/vex/functions/planesphereintersect)
+[predicate_incircle](http://www.sidefx.com/docs/houdini16.0/vex/functions/predicate_incircle)
+[predicate_insphere](http://www.sidefx.com/docs/houdini16.0/vex/functions/predicate_insphere)
+[predicate_orient2d](http://www.sidefx.com/docs/houdini16.0/vex/functions/predicate_orient2d)
+[predicate_orient3d](http://www.sidefx.com/docs/houdini16.0/vex/functions/predicate_orient3d)
+[primarclen](http://www.sidefx.com/docs/houdini16.0/vex/functions/primarclen)
+[primduv](http://www.sidefx.com/docs/houdini16.0/vex/functions/primduv)
+[primfind](http://www.sidefx.com/docs/houdini16.0/vex/functions/primfind)
+[primuvconvert](http://www.sidefx.com/docs/houdini16.0/vex/functions/primuvconvert)
+[sample_cdf](http://www.sidefx.com/docs/houdini16.0/vex/functions/sample_cdf)
+[scatter](http://www.sidefx.com/docs/houdini16.0/vex/functions/scatter)
+[setagentclips](http://www.sidefx.com/docs/houdini16.0/vex/functions/setagentclips)
+[setedgegroup](http://www.sidefx.com/docs/houdini16.0/vex/functions/setedgegroup)
+[solid_angle](http://www.sidefx.com/docs/houdini16.0/vex/functions/solid_angle)
+[split_bsdf](http://www.sidefx.com/docs/houdini16.0/vex/functions/split_bsdf)
+[sssapprox](http://www.sidefx.com/docs/houdini16.0/vex/functions/sssapprox)
+[uvdist](http://www.sidefx.com/docs/houdini16.0/vex/functions/uvdist)
+[uvintersect](http://www.sidefx.com/docs/houdini16.0/vex/functions/uvintersect)
+[uvsample](http://www.sidefx.com/docs/houdini16.0/vex/functions/uvsample)
+[volumevoxeldiameter](http://www.sidefx.com/docs/houdini16.0/vex/functions/volumevoxeldiameter)

Total: 724 functions.
    
#### New in 15.5

+[agentclipchannel](http://www.sidefx.com/docs/houdini15.5/vex/functions/agentclipchannel)
+[agentclipsample](http://www.sidefx.com/docs/houdini15.5/vex/functions/agentclipsample)
+[agentclipsamplerate](http://www.sidefx.com/docs/houdini15.5/vex/functions/agentclipsamplerate)
+[getderiv](http://www.sidefx.com/docs/houdini15.5/vex/functions/getderiv)
+[getsamplestore](http://www.sidefx.com/docs/houdini15.5/vex/functions/getsamplestore)
+[getuvobjects](http://www.sidefx.com/docs/houdini15.5/vex/functions/getuvobjects)
+[getuvtangents](http://www.sidefx.com/docs/houdini15.5/vex/functions/getuvtangents)
+[setsamplestore](http://www.sidefx.com/docs/houdini15.5/vex/functions/setsamplestore)
+[slideframe](http://www.sidefx.com/docs/houdini15.5/vex/functions/slideframe)

Total: 651 functions.
    
#### New in 15.0

-[addattribute](http://www.sidefx.com/docs/houdini14.0/vex/functions/addattribute)
-[addgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/addgroup)
-[bumpmap](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmap)
-[bumpmapA](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapA)
-[bumpmapB](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapB)
-[bumpmapG](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapG)
-[bumpmapL](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapL)
-[bumpmapR](http://www.sidefx.com/docs/houdini14.0/vex/functions/bumpmapR)
-[file_eof](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_eof)
-[file_flush](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_flush)
-[file_open](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_open)
-[file_read](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_read)
-[file_readline](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_readline)
-[file_readlines](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_readlines)
-[file_seek](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_seek)
-[file_size](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_size)
-[file_tell](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_tell)
-[file_write](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_write)
-[file_writeline](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_writeline)
-[file_writelines](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_writelines)
-[import](http://www.sidefx.com/docs/houdini14.0/vex/functions/import)
-[ingroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/ingroup)
-[newgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/newgroup)
-[rawbumpmap](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmap)
-[rawbumpmapA](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapA)
-[rawbumpmapB](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapB)
-[rawbumpmapG](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapG)
-[rawbumpmapL](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapL)
-[rawbumpmapR](http://www.sidefx.com/docs/houdini14.0/vex/functions/rawbumpmapR)
-[removegroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/removegroup)
-[variadicarguments](http://www.sidefx.com/docs/houdini14.0/vex/functions/variadicarguments)+[addvisualizer](http://www.sidefx.com/docs/houdini15.0/vex/functions/addvisualizer)
+[attrib](http://www.sidefx.com/docs/houdini15.0/vex/functions/attrib)
+[attribclass](http://www.sidefx.com/docs/houdini15.0/vex/functions/attribclass)
+[error](http://www.sidefx.com/docs/houdini15.0/vex/functions/error)
+[fuzzify](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzify)
+[fuzzy_and](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_and)
+[fuzzy_defuzz_centroid](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_defuzz_centroid)
+[fuzzy_nand](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_nand)
+[fuzzy_nor](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_nor)
+[fuzzy_not](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_not)
+[fuzzy_nxor](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_nxor)
+[fuzzy_or](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_or)
+[fuzzy_xor](http://www.sidefx.com/docs/houdini15.0/vex/functions/fuzzy_xor)
+[getattrib](http://www.sidefx.com/docs/houdini15.0/vex/functions/getattrib)
+[getsmoothP](http://www.sidefx.com/docs/houdini15.0/vex/functions/getsmoothP)
+[hasattrib](http://www.sidefx.com/docs/houdini15.0/vex/functions/hasattrib)
+[idtopoint](http://www.sidefx.com/docs/houdini15.0/vex/functions/idtopoint)
+[idtoprim](http://www.sidefx.com/docs/houdini15.0/vex/functions/idtoprim)
+[invertexgroup](http://www.sidefx.com/docs/houdini15.0/vex/functions/invertexgroup)
+[nametopoint](http://www.sidefx.com/docs/houdini15.0/vex/functions/nametopoint)
+[nametoprim](http://www.sidefx.com/docs/houdini15.0/vex/functions/nametoprim)
+[nvertices](http://www.sidefx.com/docs/houdini15.0/vex/functions/nvertices)
+[nverticesgroup](http://www.sidefx.com/docs/houdini15.0/vex/functions/nverticesgroup)
+[opdigits](http://www.sidefx.com/docs/houdini15.0/vex/functions/opdigits)
+[opfullpath](http://www.sidefx.com/docs/houdini15.0/vex/functions/opfullpath)
+[pgfind](http://www.sidefx.com/docs/houdini15.0/vex/functions/pgfind)
+[product](http://www.sidefx.com/docs/houdini15.0/vex/functions/product)
+[relativepath](http://www.sidefx.com/docs/houdini15.0/vex/functions/relativepath)
+[resample_linear](http://www.sidefx.com/docs/houdini15.0/vex/functions/resample_linear)
+[setvertexgroup](http://www.sidefx.com/docs/houdini15.0/vex/functions/setvertexgroup)
+[setvertexpoint](http://www.sidefx.com/docs/houdini15.0/vex/functions/setvertexpoint)
+[smoothrotation](http://www.sidefx.com/docs/houdini15.0/vex/functions/smoothrotation)
+[splitpath](http://www.sidefx.com/docs/houdini15.0/vex/functions/splitpath)
+[sum](http://www.sidefx.com/docs/houdini15.0/vex/functions/sum)
+[uvunwrap](http://www.sidefx.com/docs/houdini15.0/vex/functions/uvunwrap)
+[warning](http://www.sidefx.com/docs/houdini15.0/vex/functions/warning)

Total: 642 functions.
    
#### New in 14.0

+[accessframe](http://www.sidefx.com/docs/houdini14.0/vex/functions/accessframe)
+[addattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/addattrib)
+[adddetailattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/adddetailattrib)
+[addpointattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/addpointattrib)
+[addprimattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/addprimattrib)
+[addvertexattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/addvertexattrib)
+[agentaddclip](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentaddclip)
+[agentclipcatalog](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentclipcatalog)
+[agentcliplength](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentcliplength)
+[agentclipnames](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentclipnames)
+[agentclipsamplelocal](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentclipsamplelocal)
+[agentclipsampleworld](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentclipsampleworld)
+[agentcliptimes](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentcliptimes)
+[agentclipweights](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentclipweights)
+[agentcollisionlayer](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentcollisionlayer)
+[agentcurrentlayer](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentcurrentlayer)
+[agentlayerbindings](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentlayerbindings)
+[agentlayers](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentlayers)
+[agentlayershapes](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentlayershapes)
+[agentlocaltransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentlocaltransform)
+[agentlocaltransforms](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentlocaltransforms)
+[agentrigchildren](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentrigchildren)
+[agentrigfind](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentrigfind)
+[agentrigparent](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentrigparent)
+[agenttransformcount](http://www.sidefx.com/docs/houdini14.0/vex/functions/agenttransformcount)
+[agenttransformnames](http://www.sidefx.com/docs/houdini14.0/vex/functions/agenttransformnames)
+[agenttransformtolocal](http://www.sidefx.com/docs/houdini14.0/vex/functions/agenttransformtolocal)
+[agenttransformtoworld](http://www.sidefx.com/docs/houdini14.0/vex/functions/agenttransformtoworld)
+[agentworldtransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentworldtransform)
+[agentworldtransforms](http://www.sidefx.com/docs/houdini14.0/vex/functions/agentworldtransforms)
+[append](http://www.sidefx.com/docs/houdini14.0/vex/functions/append)
+[argsort](http://www.sidefx.com/docs/houdini14.0/vex/functions/argsort)
+[attribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/attribtypeinfo)
+[blackbody](http://www.sidefx.com/docs/houdini14.0/vex/functions/blackbody)
+[bouncelabel](http://www.sidefx.com/docs/houdini14.0/vex/functions/bouncelabel)
+[bouncemask](http://www.sidefx.com/docs/houdini14.0/vex/functions/bouncemask)
+[cbrt](http://www.sidefx.com/docs/houdini14.0/vex/functions/cbrt)
+[chr](http://www.sidefx.com/docs/houdini14.0/vex/functions/chr)
+[curlnoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/curlnoise)
+[curlnoise2d](http://www.sidefx.com/docs/houdini14.0/vex/functions/curlnoise2d)
+[curlxnoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/curlxnoise)
+[curlxnoise2d](http://www.sidefx.com/docs/houdini14.0/vex/functions/curlxnoise2d)
+[detailattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/detailattrib)
+[detailattribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/detailattribtypeinfo)
+[endswith](http://www.sidefx.com/docs/houdini14.0/vex/functions/endswith)
+[erf_inv](http://www.sidefx.com/docs/houdini14.0/vex/functions/erf_inv)
+[eulertoquaternion](http://www.sidefx.com/docs/houdini14.0/vex/functions/eulertoquaternion)
+[expand_udim](http://www.sidefx.com/docs/houdini14.0/vex/functions/expand_udim)
+[expandpointgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/expandpointgroup)
+[expandprimgroup](http://www.sidefx.com/docs/houdini14.0/vex/functions/expandprimgroup)
+[filamentsample](http://www.sidefx.com/docs/houdini14.0/vex/functions/filamentsample)
+[file_eof](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_eof)
+[file_flush](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_flush)
+[file_open](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_open)
+[file_read](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_read)
+[file_readline](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_readline)
+[file_readlines](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_readlines)
+[file_seek](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_seek)
+[file_size](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_size)
+[file_stat](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_stat)
+[file_tell](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_tell)
+[file_write](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_write)
+[file_writeline](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_writeline)
+[file_writelines](http://www.sidefx.com/docs/houdini14.0/vex/functions/file_writelines)
+[find](http://www.sidefx.com/docs/houdini14.0/vex/functions/find)
+[getcomponents](http://www.sidefx.com/docs/houdini14.0/vex/functions/getcomponents)
+[getlight](http://www.sidefx.com/docs/houdini14.0/vex/functions/getlight)
+[getlightid](http://www.sidefx.com/docs/houdini14.0/vex/functions/getlightid)
+[getlightscope](http://www.sidefx.com/docs/houdini14.0/vex/functions/getlightscope)
+[getmaterial](http://www.sidefx.com/docs/houdini14.0/vex/functions/getmaterial)
+[getpointbbox](http://www.sidefx.com/docs/houdini14.0/vex/functions/getpointbbox)
+[getscope](http://www.sidefx.com/docs/houdini14.0/vex/functions/getscope)
+[getspace](http://www.sidefx.com/docs/houdini14.0/vex/functions/getspace)
+[hair](http://www.sidefx.com/docs/houdini14.0/vex/functions/hair)
+[haslight](http://www.sidefx.com/docs/houdini14.0/vex/functions/haslight)
+[insert](http://www.sidefx.com/docs/houdini14.0/vex/functions/insert)
+[intersect_all](http://www.sidefx.com/docs/houdini14.0/vex/functions/intersect_all)
+[isalpha](http://www.sidefx.com/docs/houdini14.0/vex/functions/isalpha)
+[isdigit](http://www.sidefx.com/docs/houdini14.0/vex/functions/isdigit)
+[istracing](http://www.sidefx.com/docs/houdini14.0/vex/functions/istracing)
+[isuvrendering](http://www.sidefx.com/docs/houdini14.0/vex/functions/isuvrendering)
+[isvalidindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/isvalidindex)
+[join](http://www.sidefx.com/docs/houdini14.0/vex/functions/join)
+[len](http://www.sidefx.com/docs/houdini14.0/vex/functions/len)
+[lightid](http://www.sidefx.com/docs/houdini14.0/vex/functions/lightid)
+[limit_sample_space](http://www.sidefx.com/docs/houdini14.0/vex/functions/limit_sample_space)
+[lstrip](http://www.sidefx.com/docs/houdini14.0/vex/functions/lstrip)
+[makebasis](http://www.sidefx.com/docs/houdini14.0/vex/functions/makebasis)
+[nearpoints](http://www.sidefx.com/docs/houdini14.0/vex/functions/nearpoints)
+[noised](http://www.sidefx.com/docs/houdini14.0/vex/functions/noised)
+[opend](http://www.sidefx.com/docs/houdini14.0/vex/functions/opend)
+[opstart](http://www.sidefx.com/docs/houdini14.0/vex/functions/opstart)
+[ord](http://www.sidefx.com/docs/houdini14.0/vex/functions/ord)
+[osd_facecount](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_facecount)
+[osd_firstpatch](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_firstpatch)
+[osd_limitsurface](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_limitsurface)
+[osd_limitsurfacevertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_limitsurfacevertex)
+[osd_patchcount](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_patchcount)
+[osd_patches](http://www.sidefx.com/docs/houdini14.0/vex/functions/osd_patches)
+[outerproduct](http://www.sidefx.com/docs/houdini14.0/vex/functions/outerproduct)
+[pcfind](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcfind)
+[pcfind_radius](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcfind_radius)
+[pcimportbyidxv](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcimportbyidxv)
+[pcsize](http://www.sidefx.com/docs/houdini14.0/vex/functions/pcsize)
+[pointattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointattrib)
+[pointattribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointattribtypeinfo)
+[pointhedge](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointhedge)
+[pointhedgenext](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointhedgenext)
+[pointprims](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointprims)
+[pointvertices](http://www.sidefx.com/docs/houdini14.0/vex/functions/pointvertices)
+[polardecomp](http://www.sidefx.com/docs/houdini14.0/vex/functions/polardecomp)
+[primattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/primattrib)
+[primattribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/primattribtypeinfo)
+[primhedge](http://www.sidefx.com/docs/houdini14.0/vex/functions/primhedge)
+[primpoint](http://www.sidefx.com/docs/houdini14.0/vex/functions/primpoint)
+[primpoints](http://www.sidefx.com/docs/houdini14.0/vex/functions/primpoints)
+[primvertex](http://www.sidefx.com/docs/houdini14.0/vex/functions/primvertex)
+[primvertices](http://www.sidefx.com/docs/houdini14.0/vex/functions/primvertices)
+[ptexture](http://www.sidefx.com/docs/houdini14.0/vex/functions/ptexture)
+[random_fhash](http://www.sidefx.com/docs/houdini14.0/vex/functions/random_fhash)
+[random_ihash](http://www.sidefx.com/docs/houdini14.0/vex/functions/random_ihash)
+[random_shash](http://www.sidefx.com/docs/houdini14.0/vex/functions/random_shash)
+[re_find](http://www.sidefx.com/docs/houdini14.0/vex/functions/re_find)
+[re_findall](http://www.sidefx.com/docs/houdini14.0/vex/functions/re_findall)
+[re_match](http://www.sidefx.com/docs/houdini14.0/vex/functions/re_match)
+[re_replace](http://www.sidefx.com/docs/houdini14.0/vex/functions/re_replace)
+[re_split](http://www.sidefx.com/docs/houdini14.0/vex/functions/re_split)
+[relpointbbox](http://www.sidefx.com/docs/houdini14.0/vex/functions/relpointbbox)
+[removeindex](http://www.sidefx.com/docs/houdini14.0/vex/functions/removeindex)
+[removevalue](http://www.sidefx.com/docs/houdini14.0/vex/functions/removevalue)
+[reorder](http://www.sidefx.com/docs/houdini14.0/vex/functions/reorder)
+[reverse](http://www.sidefx.com/docs/houdini14.0/vex/functions/reverse)
+[rgbtoxyz](http://www.sidefx.com/docs/houdini14.0/vex/functions/rgbtoxyz)
+[rotate_x_to](http://www.sidefx.com/docs/houdini14.0/vex/functions/rotate_x_to)
+[rstrip](http://www.sidefx.com/docs/houdini14.0/vex/functions/rstrip)
+[sample_cauchy](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_cauchy)
+[sample_circle_arc](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_circle_arc)
+[sample_circle_edge_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_circle_edge_uniform)
+[sample_circle_slice](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_circle_slice)
+[sample_circle_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_circle_uniform)
+[sample_direction_cone](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_direction_cone)
+[sample_direction_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_direction_uniform)
+[sample_discrete](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_discrete)
+[sample_exponential](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_exponential)
+[sample_hemisphere](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_hemisphere)
+[sample_hypersphere_cone](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_hypersphere_cone)
+[sample_hypersphere_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_hypersphere_uniform)
+[sample_lognormal](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_lognormal)
+[sample_lognormal_by_median](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_lognormal_by_median)
+[sample_normal](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_normal)
+[sample_orientation_cone](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_orientation_cone)
+[sample_orientation_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_orientation_uniform)
+[sample_photon](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_photon)
+[sample_sphere_cone](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_sphere_cone)
+[sample_sphere_uniform](http://www.sidefx.com/docs/houdini14.0/vex/functions/sample_sphere_uniform)
+[sampledisk](http://www.sidefx.com/docs/houdini14.0/vex/functions/sampledisk)
+[select](http://www.sidefx.com/docs/houdini14.0/vex/functions/select)
+[sensor_panorama_create](http://www.sidefx.com/docs/houdini14.0/vex/functions/sensor_panorama_create)
+[sensor_panorama_getcolor](http://www.sidefx.com/docs/houdini14.0/vex/functions/sensor_panorama_getcolor)
+[sensor_panorama_getcone](http://www.sidefx.com/docs/houdini14.0/vex/functions/sensor_panorama_getcone)
+[sensor_panorama_getdepth](http://www.sidefx.com/docs/houdini14.0/vex/functions/sensor_panorama_getdepth)
+[sensor_save](http://www.sidefx.com/docs/houdini14.0/vex/functions/sensor_save)
+[setagentclipnames](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentclipnames)
+[setagentcliptimes](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentcliptimes)
+[setagentclipweights](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentclipweights)
+[setagentcollisionlayer](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentcollisionlayer)
+[setagentcurrentlayer](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentcurrentlayer)
+[setagentlocaltransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentlocaltransform)
+[setagentlocaltransforms](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentlocaltransforms)
+[setagentworldtransform](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentworldtransform)
+[setagentworldtransforms](http://www.sidefx.com/docs/houdini14.0/vex/functions/setagentworldtransforms)
+[setattribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/setattribtypeinfo)
+[sign](http://www.sidefx.com/docs/houdini14.0/vex/functions/sign)
+[slice](http://www.sidefx.com/docs/houdini14.0/vex/functions/slice)
+[solvecubic](http://www.sidefx.com/docs/houdini14.0/vex/functions/solvecubic)
+[solvepoly](http://www.sidefx.com/docs/houdini14.0/vex/functions/solvepoly)
+[solvequadratic](http://www.sidefx.com/docs/houdini14.0/vex/functions/solvequadratic)
+[sort](http://www.sidefx.com/docs/houdini14.0/vex/functions/sort)
+[startswith](http://www.sidefx.com/docs/houdini14.0/vex/functions/startswith)
+[strip](http://www.sidefx.com/docs/houdini14.0/vex/functions/strip)
+[switch](http://www.sidefx.com/docs/houdini14.0/vex/functions/switch)
+[titlecase](http://www.sidefx.com/docs/houdini14.0/vex/functions/titlecase)
+[tolower](http://www.sidefx.com/docs/houdini14.0/vex/functions/tolower)
+[toupper](http://www.sidefx.com/docs/houdini14.0/vex/functions/toupper)
+[translucent](http://www.sidefx.com/docs/houdini14.0/vex/functions/translucent)
+[upush](http://www.sidefx.com/docs/houdini14.0/vex/functions/upush)
+[vertexattrib](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexattrib)
+[vertexattribtypeinfo](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexattribtypeinfo)
+[vertexhedge](http://www.sidefx.com/docs/houdini14.0/vex/functions/vertexhedge)
+[writepixel](http://www.sidefx.com/docs/houdini14.0/vex/functions/writepixel)
+[xnoise](http://www.sidefx.com/docs/houdini14.0/vex/functions/xnoise)
+[xnoised](http://www.sidefx.com/docs/houdini14.0/vex/functions/xnoised)
+[xyztorgb](http://www.sidefx.com/docs/houdini14.0/vex/functions/xyztorgb)

Total: 637 functions.
    
#### New in 13.0

-[fntransform](http://www.sidefx.com/docs/houdini12.5/vex/functions/fntransform)
-[ftransform](http://www.sidefx.com/docs/houdini12.5/vex/functions/ftransform)
-[fvtransform](http://www.sidefx.com/docs/houdini12.5/vex/functions/fvtransform)
-[getneighbour](http://www.sidefx.com/docs/houdini12.5/vex/functions/getneighbour)
-[getneighbourcount](http://www.sidefx.com/docs/houdini12.5/vex/functions/getneighbourcount)
-[getneighbours](http://www.sidefx.com/docs/houdini12.5/vex/functions/getneighbours)
-[lntransform](http://www.sidefx.com/docs/houdini12.5/vex/functions/lntransform)
-[ltransform](http://www.sidefx.com/docs/houdini12.5/vex/functions/ltransform)
-[lvtransform](http://www.sidefx.com/docs/houdini12.5/vex/functions/lvtransform)
-[ontransform](http://www.sidefx.com/docs/houdini12.5/vex/functions/ontransform)
-[otransform](http://www.sidefx.com/docs/houdini12.5/vex/functions/otransform)
-[ovtransform](http://www.sidefx.com/docs/houdini12.5/vex/functions/ovtransform)
-[storelightexports](http://www.sidefx.com/docs/houdini12.5/vex/functions/storelightexports)+[addpoint](http://www.sidefx.com/docs/houdini13.0/vex/functions/addpoint)
+[addprim](http://www.sidefx.com/docs/houdini13.0/vex/functions/addprim)
+[addvertex](http://www.sidefx.com/docs/houdini13.0/vex/functions/addvertex)
+[assert_enabled](http://www.sidefx.com/docs/houdini13.0/vex/functions/assert_enabled)
+[attribsize](http://www.sidefx.com/docs/houdini13.0/vex/functions/attribsize)
+[attribtype](http://www.sidefx.com/docs/houdini13.0/vex/functions/attribtype)
+[ch3](http://www.sidefx.com/docs/houdini13.0/vex/functions/ch3)
+[ch4](http://www.sidefx.com/docs/houdini13.0/vex/functions/ch4)
+[chf](http://www.sidefx.com/docs/houdini13.0/vex/functions/chf)
+[chi](http://www.sidefx.com/docs/houdini13.0/vex/functions/chi)
+[chp](http://www.sidefx.com/docs/houdini13.0/vex/functions/chp)
+[chramp](http://www.sidefx.com/docs/houdini13.0/vex/functions/chramp)
+[chs](http://www.sidefx.com/docs/houdini13.0/vex/functions/chs)
+[chsraw](http://www.sidefx.com/docs/houdini13.0/vex/functions/chsraw)
+[chv](http://www.sidefx.com/docs/houdini13.0/vex/functions/chv)
+[detailattribsize](http://www.sidefx.com/docs/houdini13.0/vex/functions/detailattribsize)
+[detailattribtype](http://www.sidefx.com/docs/houdini13.0/vex/functions/detailattribtype)
+[detailintrinsic](http://www.sidefx.com/docs/houdini13.0/vex/functions/detailintrinsic)
+[eigenvalues](http://www.sidefx.com/docs/houdini13.0/vex/functions/eigenvalues)
+[findattribval](http://www.sidefx.com/docs/houdini13.0/vex/functions/findattribval)
+[findattribvalcount](http://www.sidefx.com/docs/houdini13.0/vex/functions/findattribvalcount)
+[fit01](http://www.sidefx.com/docs/houdini13.0/vex/functions/fit01)
+[fit10](http://www.sidefx.com/docs/houdini13.0/vex/functions/fit10)
+[fit11](http://www.sidefx.com/docs/houdini13.0/vex/functions/fit11)
+[geoself](http://www.sidefx.com/docs/houdini13.0/vex/functions/geoself)
+[getfogname](http://www.sidefx.com/docs/houdini13.0/vex/functions/getfogname)
+[getprimid](http://www.sidefx.com/docs/houdini13.0/vex/functions/getprimid)
+[getptextureid](http://www.sidefx.com/docs/houdini13.0/vex/functions/getptextureid)
+[hasdetailattrib](http://www.sidefx.com/docs/houdini13.0/vex/functions/hasdetailattrib)
+[haspointattrib](http://www.sidefx.com/docs/houdini13.0/vex/functions/haspointattrib)
+[hasprimattrib](http://www.sidefx.com/docs/houdini13.0/vex/functions/hasprimattrib)
+[hasvertexattrib](http://www.sidefx.com/docs/houdini13.0/vex/functions/hasvertexattrib)
+[hedge_dstpoint](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_dstpoint)
+[hedge_dstvertex](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_dstvertex)
+[hedge_equivcount](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_equivcount)
+[hedge_isequiv](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_isequiv)
+[hedge_isprimary](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_isprimary)
+[hedge_isvalid](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_isvalid)
+[hedge_next](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_next)
+[hedge_nextequiv](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_nextequiv)
+[hedge_postdstpoint](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_postdstpoint)
+[hedge_postdstvertex](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_postdstvertex)
+[hedge_presrcpoint](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_presrcpoint)
+[hedge_presrcvertex](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_presrcvertex)
+[hedge_prev](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_prev)
+[hedge_prim](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_prim)
+[hedge_primary](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_primary)
+[hedge_srcpoint](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_srcpoint)
+[hedge_srcvertex](http://www.sidefx.com/docs/houdini13.0/vex/functions/hedge_srcvertex)
+[inpointgroup](http://www.sidefx.com/docs/houdini13.0/vex/functions/inpointgroup)
+[inprimgroup](http://www.sidefx.com/docs/houdini13.0/vex/functions/inprimgroup)
+[isfinite](http://www.sidefx.com/docs/houdini13.0/vex/functions/isfinite)
+[isnan](http://www.sidefx.com/docs/houdini13.0/vex/functions/isnan)
+[isvarying](http://www.sidefx.com/docs/houdini13.0/vex/functions/isvarying)
+[itoa](http://www.sidefx.com/docs/houdini13.0/vex/functions/itoa)
+[neighbour](http://www.sidefx.com/docs/houdini13.0/vex/functions/neighbour)
+[neighbourcount](http://www.sidefx.com/docs/houdini13.0/vex/functions/neighbourcount)
+[neighbours](http://www.sidefx.com/docs/houdini13.0/vex/functions/neighbours)
+[nuniqueval](http://www.sidefx.com/docs/houdini13.0/vex/functions/nuniqueval)
+[pack_inttosafefloat](http://www.sidefx.com/docs/houdini13.0/vex/functions/pack_inttosafefloat)
+[pcimportbyidx3](http://www.sidefx.com/docs/houdini13.0/vex/functions/pcimportbyidx3)
+[pcimportbyidx4](http://www.sidefx.com/docs/houdini13.0/vex/functions/pcimportbyidx4)
+[pcimportbyidxf](http://www.sidefx.com/docs/houdini13.0/vex/functions/pcimportbyidxf)
+[pcimportbyidxi](http://www.sidefx.com/docs/houdini13.0/vex/functions/pcimportbyidxi)
+[pcimportbyidxp](http://www.sidefx.com/docs/houdini13.0/vex/functions/pcimportbyidxp)
+[pcimportbyidxs](http://www.sidefx.com/docs/houdini13.0/vex/functions/pcimportbyidxs)
+[pluralize](http://www.sidefx.com/docs/houdini13.0/vex/functions/pluralize)
+[pointattribsize](http://www.sidefx.com/docs/houdini13.0/vex/functions/pointattribsize)
+[pointattribtype](http://www.sidefx.com/docs/houdini13.0/vex/functions/pointattribtype)
+[pointedge](http://www.sidefx.com/docs/houdini13.0/vex/functions/pointedge)
+[pointvertex](http://www.sidefx.com/docs/houdini13.0/vex/functions/pointvertex)
+[primattribsize](http://www.sidefx.com/docs/houdini13.0/vex/functions/primattribsize)
+[primattribtype](http://www.sidefx.com/docs/houdini13.0/vex/functions/primattribtype)
+[primintrinsic](http://www.sidefx.com/docs/houdini13.0/vex/functions/primintrinsic)
+[primvertexcount](http://www.sidefx.com/docs/houdini13.0/vex/functions/primvertexcount)
+[print_once](http://www.sidefx.com/docs/houdini13.0/vex/functions/print_once)
+[qdistance](http://www.sidefx.com/docs/houdini13.0/vex/functions/qdistance)
+[qinvert](http://www.sidefx.com/docs/houdini13.0/vex/functions/qinvert)
+[qrotate](http://www.sidefx.com/docs/houdini13.0/vex/functions/qrotate)
+[rand](http://www.sidefx.com/docs/houdini13.0/vex/functions/rand)
+[random_sobol](http://www.sidefx.com/docs/houdini13.0/vex/functions/random_sobol)
+[removepoint](http://www.sidefx.com/docs/houdini13.0/vex/functions/removepoint)
+[removeprim](http://www.sidefx.com/docs/houdini13.0/vex/functions/removeprim)
+[setattrib](http://www.sidefx.com/docs/houdini13.0/vex/functions/setattrib)
+[setcurrentlight](http://www.sidefx.com/docs/houdini13.0/vex/functions/setcurrentlight)
+[setdetailattrib](http://www.sidefx.com/docs/houdini13.0/vex/functions/setdetailattrib)
+[setpointattrib](http://www.sidefx.com/docs/houdini13.0/vex/functions/setpointattrib)
+[setpointgroup](http://www.sidefx.com/docs/houdini13.0/vex/functions/setpointgroup)
+[setprimattrib](http://www.sidefx.com/docs/houdini13.0/vex/functions/setprimattrib)
+[setprimgroup](http://www.sidefx.com/docs/houdini13.0/vex/functions/setprimgroup)
+[setprimintrinsic](http://www.sidefx.com/docs/houdini13.0/vex/functions/setprimintrinsic)
+[setprimvertex](http://www.sidefx.com/docs/houdini13.0/vex/functions/setprimvertex)
+[setvertexattrib](http://www.sidefx.com/docs/houdini13.0/vex/functions/setvertexattrib)
+[shrz](http://www.sidefx.com/docs/houdini13.0/vex/functions/shrz)
+[split](http://www.sidefx.com/docs/houdini13.0/vex/functions/split)
+[storelightexport](http://www.sidefx.com/docs/houdini13.0/vex/functions/storelightexport)
+[tet_adjacent](http://www.sidefx.com/docs/houdini13.0/vex/functions/tet_adjacent)
+[tet_faceindex](http://www.sidefx.com/docs/houdini13.0/vex/functions/tet_faceindex)
+[texprintf](http://www.sidefx.com/docs/houdini13.0/vex/functions/texprintf)
+[uniqueval](http://www.sidefx.com/docs/houdini13.0/vex/functions/uniqueval)
+[unpack_intfromsafefloat](http://www.sidefx.com/docs/houdini13.0/vex/functions/unpack_intfromsafefloat)
+[variance](http://www.sidefx.com/docs/houdini13.0/vex/functions/variance)
+[vertexattribsize](http://www.sidefx.com/docs/houdini13.0/vex/functions/vertexattribsize)
+[vertexattribtype](http://www.sidefx.com/docs/houdini13.0/vex/functions/vertexattribtype)
+[vertexindex](http://www.sidefx.com/docs/houdini13.0/vex/functions/vertexindex)
+[vertexnext](http://www.sidefx.com/docs/houdini13.0/vex/functions/vertexnext)
+[vertexpoint](http://www.sidefx.com/docs/houdini13.0/vex/functions/vertexpoint)
+[vertexprev](http://www.sidefx.com/docs/houdini13.0/vex/functions/vertexprev)
+[vertexprim](http://www.sidefx.com/docs/houdini13.0/vex/functions/vertexprim)
+[vertexprimindex](http://www.sidefx.com/docs/houdini13.0/vex/functions/vertexprimindex)
+[volumeindex](http://www.sidefx.com/docs/houdini13.0/vex/functions/volumeindex)
+[volumeindexorigin](http://www.sidefx.com/docs/houdini13.0/vex/functions/volumeindexorigin)
+[volumeindextopos](http://www.sidefx.com/docs/houdini13.0/vex/functions/volumeindextopos)
+[volumeindexv](http://www.sidefx.com/docs/houdini13.0/vex/functions/volumeindexv)
+[volumepostoindex](http://www.sidefx.com/docs/houdini13.0/vex/functions/volumepostoindex)
+[volumeres](http://www.sidefx.com/docs/houdini13.0/vex/functions/volumeres)
+[volumesamplev](http://www.sidefx.com/docs/houdini13.0/vex/functions/volumesamplev)

Total: 444 functions.
    
#### New in 12.5

-[forpoint](http://www.sidefx.com/docs/houdini12.1/vex/functions/forpoint)+[cvex_bsdf](http://www.sidefx.com/docs/houdini12.5/vex/functions/cvex_bsdf)
+[detail](http://www.sidefx.com/docs/houdini12.5/vex/functions/detail)
+[forpoints](http://www.sidefx.com/docs/houdini12.5/vex/functions/forpoints)
+[instance](http://www.sidefx.com/docs/houdini12.5/vex/functions/instance)
+[israytracing](http://www.sidefx.com/docs/houdini12.5/vex/functions/israytracing)
+[minpos](http://www.sidefx.com/docs/houdini12.5/vex/functions/minpos)
+[nearpoint](http://www.sidefx.com/docs/houdini12.5/vex/functions/nearpoint)
+[npointsgroup](http://www.sidefx.com/docs/houdini12.5/vex/functions/npointsgroup)
+[nprimitivesgroup](http://www.sidefx.com/docs/houdini12.5/vex/functions/nprimitivesgroup)
+[pcsampleleaf](http://www.sidefx.com/docs/houdini12.5/vex/functions/pcsampleleaf)
+[point](http://www.sidefx.com/docs/houdini12.5/vex/functions/point)
+[prim](http://www.sidefx.com/docs/houdini12.5/vex/functions/prim)
+[primuv](http://www.sidefx.com/docs/houdini12.5/vex/functions/primuv)
+[vertex](http://www.sidefx.com/docs/houdini12.5/vex/functions/vertex)
+[xyzdist](http://www.sidefx.com/docs/houdini12.5/vex/functions/xyzdist)

Total: 340 functions.
    
#### New in 12.1

-[int](http://www.sidefx.com/docs/houdini12.0/vex/functions/int)+[pcnumfound](http://www.sidefx.com/docs/houdini12.1/vex/functions/pcnumfound)
+[trunc](http://www.sidefx.com/docs/houdini12.1/vex/functions/trunc)

Total: 326 functions.
    
#### New in 12.0

+[erf](http://www.sidefx.com/docs/houdini12.0/vex/functions/erf)
+[erfc](http://www.sidefx.com/docs/houdini12.0/vex/functions/erfc)
+[flownoise](http://www.sidefx.com/docs/houdini12.0/vex/functions/flownoise)
+[flowpnoise](http://www.sidefx.com/docs/houdini12.0/vex/functions/flowpnoise)
+[gradient](http://www.sidefx.com/docs/houdini12.0/vex/functions/gradient)
+[henyeygreenstein](http://www.sidefx.com/docs/houdini12.0/vex/functions/henyeygreenstein)

Total: 325 functions.
    
#### New in 11.0

-[eval_light](http://www.sidefx.com/docs/houdini10.0/vex/functions/eval_light)
-[getshadowscope](http://www.sidefx.com/docs/houdini10.0/vex/functions/getshadowscope)
-[isdistantlight](http://www.sidefx.com/docs/houdini10.0/vex/functions/isdistantlight)
-[specularratio_bsdf](http://www.sidefx.com/docs/houdini10.0/vex/functions/specularratio_bsdf)+[albedo](http://www.sidefx.com/docs/houdini11.0/vex/functions/albedo)
+[getattribute](http://www.sidefx.com/docs/houdini11.0/vex/functions/getattribute)
+[isotropic](http://www.sidefx.com/docs/houdini11.0/vex/functions/isotropic)
+[normal_bsdf](http://www.sidefx.com/docs/houdini11.0/vex/functions/normal_bsdf)
+[pcgenerate](http://www.sidefx.com/docs/houdini11.0/vex/functions/pcgenerate)
+[pcopenlod](http://www.sidefx.com/docs/houdini11.0/vex/functions/pcopenlod)
+[resolvemissedray](http://www.sidefx.com/docs/houdini11.0/vex/functions/resolvemissedray)
+[sample_geometry](http://www.sidefx.com/docs/houdini11.0/vex/functions/sample_geometry)
+[shadow_light](http://www.sidefx.com/docs/houdini11.0/vex/functions/shadow_light)
+[swizzle](http://www.sidefx.com/docs/houdini11.0/vex/functions/swizzle)
+[variadicarguments](http://www.sidefx.com/docs/houdini11.0/vex/functions/variadicarguments)

Total: 319 functions.
    
#### From 10.0

[Du](http://www.sidefx.com/docs/houdini10.0/vex/functions/Du)
[Dv](http://www.sidefx.com/docs/houdini10.0/vex/functions/Dv)
[Dw](http://www.sidefx.com/docs/houdini10.0/vex/functions/Dw)
[abs](http://www.sidefx.com/docs/houdini10.0/vex/functions/abs)
[acos](http://www.sidefx.com/docs/houdini10.0/vex/functions/acos)
[addvariablename](http://www.sidefx.com/docs/houdini10.0/vex/functions/addvariablename)
[alphaname](http://www.sidefx.com/docs/houdini10.0/vex/functions/alphaname)
[ambient](http://www.sidefx.com/docs/houdini10.0/vex/functions/ambient)
[anoise](http://www.sidefx.com/docs/houdini10.0/vex/functions/anoise)
[area](http://www.sidefx.com/docs/houdini10.0/vex/functions/area)
[array](http://www.sidefx.com/docs/houdini10.0/vex/functions/array)
[arraylength](http://www.sidefx.com/docs/houdini10.0/vex/functions/arraylength)
[ashikhmin](http://www.sidefx.com/docs/houdini10.0/vex/functions/ashikhmin)
[asin](http://www.sidefx.com/docs/houdini10.0/vex/functions/asin)
[assign](http://www.sidefx.com/docs/houdini10.0/vex/functions/assign)
[atan](http://www.sidefx.com/docs/houdini10.0/vex/functions/atan)
[atan2](http://www.sidefx.com/docs/houdini10.0/vex/functions/atan2)
[atof](http://www.sidefx.com/docs/houdini10.0/vex/functions/atof)
[atoi](http://www.sidefx.com/docs/houdini10.0/vex/functions/atoi)
[atten](http://www.sidefx.com/docs/houdini10.0/vex/functions/atten)
[avg](http://www.sidefx.com/docs/houdini10.0/vex/functions/avg)
[binput](http://www.sidefx.com/docs/houdini10.0/vex/functions/binput)
[blinn](http://www.sidefx.com/docs/houdini10.0/vex/functions/blinn)
[blinnBRDF](http://www.sidefx.com/docs/houdini10.0/vex/functions/blinnBRDF)
[bumpname](http://www.sidefx.com/docs/houdini10.0/vex/functions/bumpname)
[ceil](http://www.sidefx.com/docs/houdini10.0/vex/functions/ceil)
[ch](http://www.sidefx.com/docs/houdini10.0/vex/functions/ch)
[chend](http://www.sidefx.com/docs/houdini10.0/vex/functions/chend)
[chendf](http://www.sidefx.com/docs/houdini10.0/vex/functions/chendf)
[chendt](http://www.sidefx.com/docs/houdini10.0/vex/functions/chendt)
[chinput](http://www.sidefx.com/docs/houdini10.0/vex/functions/chinput)
[chname](http://www.sidefx.com/docs/houdini10.0/vex/functions/chname)
[chnumchan](http://www.sidefx.com/docs/houdini10.0/vex/functions/chnumchan)
[chrate](http://www.sidefx.com/docs/houdini10.0/vex/functions/chrate)
[chstart](http://www.sidefx.com/docs/houdini10.0/vex/functions/chstart)
[chstartf](http://www.sidefx.com/docs/houdini10.0/vex/functions/chstartf)
[chstartt](http://www.sidefx.com/docs/houdini10.0/vex/functions/chstartt)
[cinput](http://www.sidefx.com/docs/houdini10.0/vex/functions/cinput)
[ckspline](http://www.sidefx.com/docs/houdini10.0/vex/functions/ckspline)
[clamp](http://www.sidefx.com/docs/houdini10.0/vex/functions/clamp)
[clip](http://www.sidefx.com/docs/houdini10.0/vex/functions/clip)
[colormap](http://www.sidefx.com/docs/houdini10.0/vex/functions/colormap)
[colorname](http://www.sidefx.com/docs/houdini10.0/vex/functions/colorname)
[computenormal](http://www.sidefx.com/docs/houdini10.0/vex/functions/computenormal)
[concat](http://www.sidefx.com/docs/houdini10.0/vex/functions/concat)
[cone](http://www.sidefx.com/docs/houdini10.0/vex/functions/cone)
[cos](http://www.sidefx.com/docs/houdini10.0/vex/functions/cos)
[cosh](http://www.sidefx.com/docs/houdini10.0/vex/functions/cosh)
[cracktransform](http://www.sidefx.com/docs/houdini10.0/vex/functions/cracktransform)
[cross](http://www.sidefx.com/docs/houdini10.0/vex/functions/cross)
[cspline](http://www.sidefx.com/docs/houdini10.0/vex/functions/cspline)
[ctransform](http://www.sidefx.com/docs/houdini10.0/vex/functions/ctransform)
[degrees](http://www.sidefx.com/docs/houdini10.0/vex/functions/degrees)
[depthmap](http://www.sidefx.com/docs/houdini10.0/vex/functions/depthmap)
[depthname](http://www.sidefx.com/docs/houdini10.0/vex/functions/depthname)
[determinant](http://www.sidefx.com/docs/houdini10.0/vex/functions/determinant)
[diffuse](http://www.sidefx.com/docs/houdini10.0/vex/functions/diffuse)
[diffuseBRDF](http://www.sidefx.com/docs/houdini10.0/vex/functions/diffuseBRDF)
[dihedral](http://www.sidefx.com/docs/houdini10.0/vex/functions/dihedral)
[dimport](http://www.sidefx.com/docs/houdini10.0/vex/functions/dimport)
[distance](http://www.sidefx.com/docs/houdini10.0/vex/functions/distance)
[distance2](http://www.sidefx.com/docs/houdini10.0/vex/functions/distance2)
[dot](http://www.sidefx.com/docs/houdini10.0/vex/functions/dot)
[dsmpixel](http://www.sidefx.com/docs/houdini10.0/vex/functions/dsmpixel)
[emission_bsdf](http://www.sidefx.com/docs/houdini10.0/vex/functions/emission_bsdf)
[environment](http://www.sidefx.com/docs/houdini10.0/vex/functions/environment)
[eval_bsdf](http://www.sidefx.com/docs/houdini10.0/vex/functions/eval_bsdf)
[exp](http://www.sidefx.com/docs/houdini10.0/vex/functions/exp)
[fastshadow](http://www.sidefx.com/docs/houdini10.0/vex/functions/fastshadow)
[filtershadow](http://www.sidefx.com/docs/houdini10.0/vex/functions/filtershadow)
[filterstep](http://www.sidefx.com/docs/houdini10.0/vex/functions/filterstep)
[finput](http://www.sidefx.com/docs/houdini10.0/vex/functions/finput)
[fit](http://www.sidefx.com/docs/houdini10.0/vex/functions/fit)
[floor](http://www.sidefx.com/docs/houdini10.0/vex/functions/floor)
[foreach](http://www.sidefx.com/docs/houdini10.0/vex/functions/foreach)
[frac](http://www.sidefx.com/docs/houdini10.0/vex/functions/frac)
[fresnel](http://www.sidefx.com/docs/houdini10.0/vex/functions/fresnel)
[fromNDC](http://www.sidefx.com/docs/houdini10.0/vex/functions/fromNDC)
[frontface](http://www.sidefx.com/docs/houdini10.0/vex/functions/frontface)
[gather](http://www.sidefx.com/docs/houdini10.0/vex/functions/gather)
[getbbox](http://www.sidefx.com/docs/houdini10.0/vex/functions/getbbox)
[getblurP](http://www.sidefx.com/docs/houdini10.0/vex/functions/getblurP)
[getbounces](http://www.sidefx.com/docs/houdini10.0/vex/functions/getbounces)
[getbounds](http://www.sidefx.com/docs/houdini10.0/vex/functions/getbounds)
[getcomp](http://www.sidefx.com/docs/houdini10.0/vex/functions/getcomp)
[getglobalraylevel](http://www.sidefx.com/docs/houdini10.0/vex/functions/getglobalraylevel)
[getlightname](http://www.sidefx.com/docs/houdini10.0/vex/functions/getlightname)
[getlights](http://www.sidefx.com/docs/houdini10.0/vex/functions/getlights)
[getobjectname](http://www.sidefx.com/docs/houdini10.0/vex/functions/getobjectname)
[getphotonlight](http://www.sidefx.com/docs/houdini10.0/vex/functions/getphotonlight)
[getraylevel](http://www.sidefx.com/docs/houdini10.0/vex/functions/getraylevel)
[getrayweight](http://www.sidefx.com/docs/houdini10.0/vex/functions/getrayweight)
[gradient3d](http://www.sidefx.com/docs/houdini10.0/vex/functions/gradient3d)
[hasplane](http://www.sidefx.com/docs/houdini10.0/vex/functions/hasplane)
[hscript_noise](http://www.sidefx.com/docs/houdini10.0/vex/functions/hscript_noise)
[hscript_rand](http://www.sidefx.com/docs/houdini10.0/vex/functions/hscript_rand)
[hscript_snoise](http://www.sidefx.com/docs/houdini10.0/vex/functions/hscript_snoise)
[hscript_sturb](http://www.sidefx.com/docs/houdini10.0/vex/functions/hscript_sturb)
[hscript_turb](http://www.sidefx.com/docs/houdini10.0/vex/functions/hscript_turb)
[hsvtorgb](http://www.sidefx.com/docs/houdini10.0/vex/functions/hsvtorgb)
[iaspect](http://www.sidefx.com/docs/houdini10.0/vex/functions/iaspect)
[ichname](http://www.sidefx.com/docs/houdini10.0/vex/functions/ichname)
[ident](http://www.sidefx.com/docs/houdini10.0/vex/functions/ident)
[iend](http://www.sidefx.com/docs/houdini10.0/vex/functions/iend)
[iendtime](http://www.sidefx.com/docs/houdini10.0/vex/functions/iendtime)
[ihasplane](http://www.sidefx.com/docs/houdini10.0/vex/functions/ihasplane)
[illuminance](http://www.sidefx.com/docs/houdini10.0/vex/functions/illuminance)
[importance_light](http://www.sidefx.com/docs/houdini10.0/vex/functions/importance_light)
[integrate3d](http://www.sidefx.com/docs/houdini10.0/vex/functions/integrate3d)
[integrate3dClip](http://www.sidefx.com/docs/houdini10.0/vex/functions/integrate3dClip)
[interpolate](http://www.sidefx.com/docs/houdini10.0/vex/functions/interpolate)
[intersect](http://www.sidefx.com/docs/houdini10.0/vex/functions/intersect)
[intersect3d](http://www.sidefx.com/docs/houdini10.0/vex/functions/intersect3d)
[intersect_lights](http://www.sidefx.com/docs/houdini10.0/vex/functions/intersect_lights)
[inumplanes](http://www.sidefx.com/docs/houdini10.0/vex/functions/inumplanes)
[invert](http://www.sidefx.com/docs/houdini10.0/vex/functions/invert)
[iplaneindex](http://www.sidefx.com/docs/houdini10.0/vex/functions/iplaneindex)
[iplanename](http://www.sidefx.com/docs/houdini10.0/vex/functions/iplanename)
[iplanesize](http://www.sidefx.com/docs/houdini10.0/vex/functions/iplanesize)
[irate](http://www.sidefx.com/docs/houdini10.0/vex/functions/irate)
[irradiance](http://www.sidefx.com/docs/houdini10.0/vex/functions/irradiance)
[isbound](http://www.sidefx.com/docs/houdini10.0/vex/functions/isbound)
[isconnected](http://www.sidefx.com/docs/houdini10.0/vex/functions/isconnected)
[isfogray](http://www.sidefx.com/docs/houdini10.0/vex/functions/isfogray)
[isframes](http://www.sidefx.com/docs/houdini10.0/vex/functions/isframes)
[issamples](http://www.sidefx.com/docs/houdini10.0/vex/functions/issamples)
[isseconds](http://www.sidefx.com/docs/houdini10.0/vex/functions/isseconds)
[isshadowray](http://www.sidefx.com/docs/houdini10.0/vex/functions/isshadowray)
[istart](http://www.sidefx.com/docs/houdini10.0/vex/functions/istart)
[istarttime](http://www.sidefx.com/docs/houdini10.0/vex/functions/istarttime)
[ixres](http://www.sidefx.com/docs/houdini10.0/vex/functions/ixres)
[iyres](http://www.sidefx.com/docs/houdini10.0/vex/functions/iyres)
[kspline](http://www.sidefx.com/docs/houdini10.0/vex/functions/kspline)
[length](http://www.sidefx.com/docs/houdini10.0/vex/functions/length)
[length2](http://www.sidefx.com/docs/houdini10.0/vex/functions/length2)
[lerp](http://www.sidefx.com/docs/houdini10.0/vex/functions/lerp)
[limport](http://www.sidefx.com/docs/houdini10.0/vex/functions/limport)
[lkspline](http://www.sidefx.com/docs/houdini10.0/vex/functions/lkspline)
[log](http://www.sidefx.com/docs/houdini10.0/vex/functions/log)
[log10](http://www.sidefx.com/docs/houdini10.0/vex/functions/log10)
[lookat](http://www.sidefx.com/docs/houdini10.0/vex/functions/lookat)
[lspline](http://www.sidefx.com/docs/houdini10.0/vex/functions/lspline)
[luminance](http://www.sidefx.com/docs/houdini10.0/vex/functions/luminance)
[lumname](http://www.sidefx.com/docs/houdini10.0/vex/functions/lumname)
[maketransform](http://www.sidefx.com/docs/houdini10.0/vex/functions/maketransform)
[maskname](http://www.sidefx.com/docs/houdini10.0/vex/functions/maskname)
[match](http://www.sidefx.com/docs/houdini10.0/vex/functions/match)
[matchvex_blinn](http://www.sidefx.com/docs/houdini10.0/vex/functions/matchvex_blinn)
[matchvex_specular](http://www.sidefx.com/docs/houdini10.0/vex/functions/matchvex_specular)
[mattrib](http://www.sidefx.com/docs/houdini10.0/vex/functions/mattrib)
[max](http://www.sidefx.com/docs/houdini10.0/vex/functions/max)
[mdensity](http://www.sidefx.com/docs/houdini10.0/vex/functions/mdensity)
[metaimport](http://www.sidefx.com/docs/houdini10.0/vex/functions/metaimport)
[metamarch](http://www.sidefx.com/docs/houdini10.0/vex/functions/metamarch)
[metanext](http://www.sidefx.com/docs/houdini10.0/vex/functions/metanext)
[metastart](http://www.sidefx.com/docs/houdini10.0/vex/functions/metastart)
[metaweight](http://www.sidefx.com/docs/houdini10.0/vex/functions/metaweight)
[min](http://www.sidefx.com/docs/houdini10.0/vex/functions/min)
[mspace](http://www.sidefx.com/docs/houdini10.0/vex/functions/mspace)
[nbouncetypes](http://www.sidefx.com/docs/houdini10.0/vex/functions/nbouncetypes)
[newsampler](http://www.sidefx.com/docs/houdini10.0/vex/functions/newsampler)
[nextsample](http://www.sidefx.com/docs/houdini10.0/vex/functions/nextsample)
[ninput](http://www.sidefx.com/docs/houdini10.0/vex/functions/ninput)
[noise](http://www.sidefx.com/docs/houdini10.0/vex/functions/noise)
[normalize](http://www.sidefx.com/docs/houdini10.0/vex/functions/normalize)
[normalname](http://www.sidefx.com/docs/houdini10.0/vex/functions/normalname)
[npoints](http://www.sidefx.com/docs/houdini10.0/vex/functions/npoints)
[nprimitives](http://www.sidefx.com/docs/houdini10.0/vex/functions/nprimitives)
[nrandom](http://www.sidefx.com/docs/houdini10.0/vex/functions/nrandom)
[ntransform](http://www.sidefx.com/docs/houdini10.0/vex/functions/ntransform)
[occlusion](http://www.sidefx.com/docs/houdini10.0/vex/functions/occlusion)
[onoise](http://www.sidefx.com/docs/houdini10.0/vex/functions/onoise)
[optransform](http://www.sidefx.com/docs/houdini10.0/vex/functions/optransform)
[ow_nspace](http://www.sidefx.com/docs/houdini10.0/vex/functions/ow_nspace)
[ow_space](http://www.sidefx.com/docs/houdini10.0/vex/functions/ow_space)
[ow_vspace](http://www.sidefx.com/docs/houdini10.0/vex/functions/ow_vspace)
[pathtrace](http://www.sidefx.com/docs/houdini10.0/vex/functions/pathtrace)
[pcclose](http://www.sidefx.com/docs/houdini10.0/vex/functions/pcclose)
[pcconvex](http://www.sidefx.com/docs/houdini10.0/vex/functions/pcconvex)
[pcexport](http://www.sidefx.com/docs/houdini10.0/vex/functions/pcexport)
[pcfarthest](http://www.sidefx.com/docs/houdini10.0/vex/functions/pcfarthest)
[pcfilter](http://www.sidefx.com/docs/houdini10.0/vex/functions/pcfilter)
[pcimport](http://www.sidefx.com/docs/houdini10.0/vex/functions/pcimport)
[pciterate](http://www.sidefx.com/docs/houdini10.0/vex/functions/pciterate)
[pcopen](http://www.sidefx.com/docs/houdini10.0/vex/functions/pcopen)
[pcunshaded](http://www.sidefx.com/docs/houdini10.0/vex/functions/pcunshaded)
[pcwrite](http://www.sidefx.com/docs/houdini10.0/vex/functions/pcwrite)
[phong](http://www.sidefx.com/docs/houdini10.0/vex/functions/phong)
[phongBRDF](http://www.sidefx.com/docs/houdini10.0/vex/functions/phongBRDF)
[phonglobe](http://www.sidefx.com/docs/houdini10.0/vex/functions/phonglobe)
[photonmap](http://www.sidefx.com/docs/houdini10.0/vex/functions/photonmap)
[planeindex](http://www.sidefx.com/docs/houdini10.0/vex/functions/planeindex)
[planename](http://www.sidefx.com/docs/houdini10.0/vex/functions/planename)
[planesize](http://www.sidefx.com/docs/houdini10.0/vex/functions/planesize)
[pnoise](http://www.sidefx.com/docs/houdini10.0/vex/functions/pnoise)
[pointname](http://www.sidefx.com/docs/houdini10.0/vex/functions/pointname)
[pop](http://www.sidefx.com/docs/houdini10.0/vex/functions/pop)
[pow](http://www.sidefx.com/docs/houdini10.0/vex/functions/pow)
[prim_attribute](http://www.sidefx.com/docs/houdini10.0/vex/functions/prim_attribute)
[prim_normal](http://www.sidefx.com/docs/houdini10.0/vex/functions/prim_normal)
[printf](http://www.sidefx.com/docs/houdini10.0/vex/functions/printf)
[ptlined](http://www.sidefx.com/docs/houdini10.0/vex/functions/ptlined)
[ptransform](http://www.sidefx.com/docs/houdini10.0/vex/functions/ptransform)
[push](http://www.sidefx.com/docs/houdini10.0/vex/functions/push)
[qconvert](http://www.sidefx.com/docs/houdini10.0/vex/functions/qconvert)
[qmultiply](http://www.sidefx.com/docs/houdini10.0/vex/functions/qmultiply)
[quaternion](http://www.sidefx.com/docs/houdini10.0/vex/functions/quaternion)
[radians](http://www.sidefx.com/docs/houdini10.0/vex/functions/radians)
[random](http://www.sidefx.com/docs/houdini10.0/vex/functions/random)
[rawcolormap](http://www.sidefx.com/docs/houdini10.0/vex/functions/rawcolormap)
[rayhittest](http://www.sidefx.com/docs/houdini10.0/vex/functions/rayhittest)
[rayimport](http://www.sidefx.com/docs/houdini10.0/vex/functions/rayimport)
[reflect](http://www.sidefx.com/docs/houdini10.0/vex/functions/reflect)
[reflectlight](http://www.sidefx.com/docs/houdini10.0/vex/functions/reflectlight)
[refract](http://www.sidefx.com/docs/houdini10.0/vex/functions/refract)
[refractlight](http://www.sidefx.com/docs/houdini10.0/vex/functions/refractlight)
[relbbox](http://www.sidefx.com/docs/houdini10.0/vex/functions/relbbox)
[renderstate](http://www.sidefx.com/docs/houdini10.0/vex/functions/renderstate)
[resize](http://www.sidefx.com/docs/houdini10.0/vex/functions/resize)
[rgbtohsv](http://www.sidefx.com/docs/houdini10.0/vex/functions/rgbtohsv)
[rint](http://www.sidefx.com/docs/houdini10.0/vex/functions/rint)
[rotate](http://www.sidefx.com/docs/houdini10.0/vex/functions/rotate)
[sample_bsdf](http://www.sidefx.com/docs/houdini10.0/vex/functions/sample_bsdf)
[sample_light](http://www.sidefx.com/docs/houdini10.0/vex/functions/sample_light)
[scale](http://www.sidefx.com/docs/houdini10.0/vex/functions/scale)
[serialize](http://www.sidefx.com/docs/houdini10.0/vex/functions/serialize)
[set](http://www.sidefx.com/docs/houdini10.0/vex/functions/set)
[setcomp](http://www.sidefx.com/docs/houdini10.0/vex/functions/setcomp)
[shadow](http://www.sidefx.com/docs/houdini10.0/vex/functions/shadow)
[shadowmap](http://www.sidefx.com/docs/houdini10.0/vex/functions/shadowmap)
[shimport](http://www.sidefx.com/docs/houdini10.0/vex/functions/shimport)
[shl](http://www.sidefx.com/docs/houdini10.0/vex/functions/shl)
[shr](http://www.sidefx.com/docs/houdini10.0/vex/functions/shr)
[simport](http://www.sidefx.com/docs/houdini10.0/vex/functions/simport)
[sin](http://www.sidefx.com/docs/houdini10.0/vex/functions/sin)
[sinh](http://www.sidefx.com/docs/houdini10.0/vex/functions/sinh)
[sleep](http://www.sidefx.com/docs/houdini10.0/vex/functions/sleep)
[slerp](http://www.sidefx.com/docs/houdini10.0/vex/functions/slerp)
[smooth](http://www.sidefx.com/docs/houdini10.0/vex/functions/smooth)
[snoise](http://www.sidefx.com/docs/houdini10.0/vex/functions/snoise)
[specular](http://www.sidefx.com/docs/houdini10.0/vex/functions/specular)
[specularBRDF](http://www.sidefx.com/docs/houdini10.0/vex/functions/specularBRDF)
[spline](http://www.sidefx.com/docs/houdini10.0/vex/functions/spline)
[sprintf](http://www.sidefx.com/docs/houdini10.0/vex/functions/sprintf)
[sqrt](http://www.sidefx.com/docs/houdini10.0/vex/functions/sqrt)
[strlen](http://www.sidefx.com/docs/houdini10.0/vex/functions/strlen)
[tan](http://www.sidefx.com/docs/houdini10.0/vex/functions/tan)
[tanh](http://www.sidefx.com/docs/houdini10.0/vex/functions/tanh)
[teximport](http://www.sidefx.com/docs/houdini10.0/vex/functions/teximport)
[texture](http://www.sidefx.com/docs/houdini10.0/vex/functions/texture)
[texture3d](http://www.sidefx.com/docs/houdini10.0/vex/functions/texture3d)
[texture3dBox](http://www.sidefx.com/docs/houdini10.0/vex/functions/texture3dBox)
[toNDC](http://www.sidefx.com/docs/houdini10.0/vex/functions/toNDC)
[trace](http://www.sidefx.com/docs/houdini10.0/vex/functions/trace)
[translate](http://www.sidefx.com/docs/houdini10.0/vex/functions/translate)
[transpose](http://www.sidefx.com/docs/houdini10.0/vex/functions/transpose)
[tw_nspace](http://www.sidefx.com/docs/houdini10.0/vex/functions/tw_nspace)
[tw_space](http://www.sidefx.com/docs/houdini10.0/vex/functions/tw_space)
[tw_vspace](http://www.sidefx.com/docs/houdini10.0/vex/functions/tw_vspace)
[unserialize](http://www.sidefx.com/docs/houdini10.0/vex/functions/unserialize)
[velocityname](http://www.sidefx.com/docs/houdini10.0/vex/functions/velocityname)
[vnoise](http://www.sidefx.com/docs/houdini10.0/vex/functions/vnoise)
[volume](http://www.sidefx.com/docs/houdini10.0/vex/functions/volume)
[volumegradient](http://www.sidefx.com/docs/houdini10.0/vex/functions/volumegradient)
[volumesample](http://www.sidefx.com/docs/houdini10.0/vex/functions/volumesample)
[vtransform](http://www.sidefx.com/docs/houdini10.0/vex/functions/vtransform)
[wireblinn](http://www.sidefx.com/docs/houdini10.0/vex/functions/wireblinn)
[wirediffuse](http://www.sidefx.com/docs/houdini10.0/vex/functions/wirediffuse)
[wnoise](http://www.sidefx.com/docs/houdini10.0/vex/functions/wnoise)
[wo_nspace](http://www.sidefx.com/docs/houdini10.0/vex/functions/wo_nspace)
[wo_space](http://www.sidefx.com/docs/houdini10.0/vex/functions/wo_space)
[wo_vspace](http://www.sidefx.com/docs/houdini10.0/vex/functions/wo_vspace)
[wt_nspace](http://www.sidefx.com/docs/houdini10.0/vex/functions/wt_nspace)
[wt_space](http://www.sidefx.com/docs/houdini10.0/vex/functions/wt_space)
[wt_vspace](http://www.sidefx.com/docs/houdini10.0/vex/functions/wt_vspace)

Total: 275 functions.